# <b> CHEE 494: Hydrate Cage Identification Algorithm </b>

    Prepared by: Nicolas Smits

    CHEE 494:  Research Project & Seminar 1 
    Department of Chemical engineering 
    McGill University

    2024.07.16

## <b><u> Introduction and Background </u></b>

### <b> Definition of Gas Hydrates </b>
Gas hydrates, also known as clathrate hydrates, are crystalline water-based solids that are physically similar to ice. However, unlike ice they form under specific conditions of low temperature (typically below 0°C / 273K) and high pressure (NUMBER), commonly found in deep ocean sediments and permafrost regions (https://doi.org/10.1021/ie00019a001) (https://water.lsbu.ac.uk/water/clathrate_hydrates.html#:~:text=Clathrate%20hydrate%20crystalline%20ices%20b,pressure%20raises%20the%20melting%20point). Gas hydrates form polyhedron "cage" structures via hydrogen bonds between collections of water molecules. These structures encapsulate small non-polar gas molecules like methane, ethane, propane, carbon dioxide, or hydrogen sulfide within cages of hydrogen-bonded water molecules. This encapsulation property allows for a large volume of guest gas to be stored within a smaller volume of host hydrate, subsequently leading to the material's use in fields including gas separation, CO<sub>2</sub> storage, energy storage, seawater desalination and energy storage. The stability of gas hydrates depends on temperature, pressure, and the type of gas involved, with different structures (I, II, and H) forming based on the size of the encapsulated molecules (https://doi.org/10.1002/bbpc.19540580105). Without the presence of either host molecules or the correct conditions, gas hydrates decompose to crystalline ice or liquid water. Clathrates have been found in nature in large quantities, with approximately 6.4 trillion tonnes of methane sequestered in deposits located on the deep ocean floor including the Norwegian Continental Shelf or permafrost regions of the northwestern Canadian arctic (https://doi.org/10.1016/j.epsl.2004.09.005). Consequently, these are seen as a potential future energy resource due to their high methane content, but they also pose climate risks as their destabilization could release methane, a potent greenhouse gas (https://doi.org/10.1016/j.apenergy.2014.12.061). 

### <b> Gas Hydrate Geometries </b>
Gas hydrates form structures which are classified into 3 types: I, II and H, all of which are crystallographic cubic structures. Structures I and II are commonly referred to as sI and sII. The figure below summarizes the different categorizations for each cage type.

[![App Platorm](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ab/HydrateStructures.jpg/440px-HydrateStructures.jpg)](https://en.wikipedia.org/wiki/Clathrate_hydrate#cite_note-16)

(https://en.wikipedia.org/wiki/Clathrate_hydrate#cite_note-16)

#### Cage Types
The type categorization arises from the number of water molecules in the unit cell. Type I consists of a unit cell 46 water molecules. Type II unit cells are formed from 136 water molecules. Type H is made up of 36 water molecules in the unit cell.
(https://en.wikipedia.org/wiki/Clathrate_hydrate#cite_note-16)

##### Type I
Type I structures are made up of two distinct cages. The small cage structure, also common to all 3 types, is a pentagonal dodecahedron known as the 5<sup>12</sup> structure (left-hand side of each row in figure). The large structure associated with this type is (and only this type) is a tetradecahedron formed from 2 hexagonal and 12 pentagonal faces, known as a 6<sup>2</sup>5<sup>12</sup> (right-most of first row in figure). A typical unit cell consists of two small and six large cages which can be replicated to form larger repetitions of the structure. Type I structures typically host CO<sub>2</sub> or CH<sub>4</sub> molecules.
(https://en.wikipedia.org/wiki/Clathrate_hydrate#cite_note-16)

##### Type II
Type II structures are made up of two distinct cages, small and large, much like Type I. This structure consists of sixteen small and eight large cages. The small cage is the 5<sup>12</sup> cage, equivalent to Type I, while the large cage is a hexadecahedron, known as 6<sup>4</sup>5<sup>12</sup> (middle of 2nd row in figure), consisting of 4 hexagonal and 12 pentagonal faces. Type II hydrates typically host gases such as oxygen, O<sub>2</sub>, and nitrogen, N<sub>2</sub>.
(https://en.wikipedia.org/wiki/Clathrate_hydrate#cite_note-16)

##### Type H 
Type H structures are made of a small cage, an intermediate cage and a "huge "cage. The small cage is the 5<sup>12</sup> cage. The intermediate cage is a 6<sup>3</sup>5<sup>6</sup>4<sup>3</sup> structure formed from 3 square, 6 pentagonal, and 3 hexagonal faces (middle of bottom row). The final "huge" cage in the Type H structure consists of a 6<sup>8</sup>5<sup>12</sup> cage, with the corresponding number of pentagonal and hexagonal faces (left-most of bottom row in figure). Unlike Types I and II, Type H requires the cooperation of two guest gas molecules, one large and one small, while the "huge" cage allows for large molecules such as butane to be hosted. The gasses filling the smaller cages support the larger cavities.
(https://en.wikipedia.org/wiki/Clathrate_hydrate#cite_note-16)  
  
The figure below provides a visual representation of how the various structure types are constructed from the cages which make them up.

[![App Platorm](https://ars.els-cdn.com/content/image/1-s2.0-S0022024822003797-gr1.jpg)](https://www.sciencedirect.com/science/article/pii/S0022024822003797)

(https://www.sciencedirect.com/science/article/pii/S0022024822003797)

### <b> Gas Hydrate Formation </b>
Hydrate nucleation typically occurs at the vapor–liquid interface, molecular models have concentrated on this surface. The earliest model, the labile cluster mechanism, describes unstable clusters of a guest molecule surrounded by 20 to 28 water molecules (depending on the hydrate structure). These clusters can form on either side of the interface and grow to a critical size, initiating nucleation. A modified labile cluster model involves adsorption and clustering on the vapor side of the interface. Another recent mechanism, the local structuring hypothesis, focuses on the arrangement of guest molecules and hydrogen-bonded water networks similar to hydrates.

It is possible that hydrate nucleation involves a combination of these mechanisms and is analogous to ice formation, as suggested by MD simulations showing long-lived hydrogen bonds forming a compact nucleus, which then expands and freezes the system. 

#### The Labile Cluster Hypothesis

The labile cluster nucleation hypothesis proposes that water clusters around dissolved gas molecules can grow to a critical radius. The pertinent stages of this nucleation process comprise several conceptual stages. Initially, pure water has transient, labile ring structures of pentagons and hexagons. Then water forms clusters around dissolved gases, with the cluster size depending on the gas type. Coordination numbers for natural gas components vary, e.g., methane (20), ethane (24), propane (28), etc (https://doi.org/10.1201/9781420008494). These clusters combine to form different unit cells, depending on the coordination of the cluster formed around guest molecules (https://doi.org/10.1201/9781420008494). Once a critical radius has been achieved, steady growth will begin (https://www.researchgate.net/publication/237824099_Hydrate_Formation_and_CH_4_Production_from_Natural_Gas_Hydrates_-_Emphasis_on_Boundary_Conditions_and_Production_Methods). However, several factors hinder the nucleation kinetics before a critical radius is achieved due to the increase in activation barrier energy. The main kinetics hindering effect present is the transformation of different structure types transforming into one or the other following cluster formation. Additionally, this transformation process allows for different structure formation pathways which increases the barrier to nucleation (though this has been scrutinized) (https://doi.org/10.1201/9781420008494). The transformation process is influenced by the gas type. For example, ethane's transformation of empty cavities is rapid, while for propane, it is slower due to fewer cavities.

[![App Platorm](https://www.researchgate.net/profile/Knut-Arne-Birkedal/publication/237824099/figure/fig5/AS:669345986408490@1536595927741/Labile-cluster-nucleation-hypothesis-Modified-from-Sloan-and-Koh-2008-A-Clustered.ppm)](https://www.researchgate.net/figure/Labile-cluster-nucleation-hypothesis-Modified-from-Sloan-and-Koh-2008-A-Clustered_fig5_237824099)

(https://www.researchgate.net/figure/Labile-cluster-nucleation-hypothesis-Modified-from-Sloan-and-Koh-2008-A-Clustered_fig5_237824099)

#### Nucleation at the Interface Hypothesis

The nucleation at the interface hypothesis is a theory of gas hydrate formation proposed by Long and Kvamme based on the idea that nucleation occurs at the vapor side of the interface (https://doi.org/10.1201/9781420008494). Thus, this suggests that the nucleation is heterogeneous and consists of several stages. Gas molecules move to the interface at a characteristic rate of 10<sup>22</sup> molecules/cm<sup>2</sup>s, indicating the transport of molecules through a stagnant boundary. Next, gas adsorbs onto the aqueous surface into partially formed cavities. Following adsorption, the gas migrates to suitable sites on the surface where water molecules form partial and then complete cages around the adsorbed gas. Labile clusters grow on the vapor side until they reach a critical size, either by adding water and gas molecules to existing cavities or by joining cavities (https://doi.org/10.1201/9781420008494).

The hypothesis envisions a dynamic process with numerous clusters forming and dissolving rather than an orderly progression from small clusters to large hydrate masses (https://doi.org/10.1201/9781420008494).

[![App Platorm](https://www.researchgate.net/profile/Knut-Arne-Birkedal/publication/237824099/figure/fig6/AS:669345986408494@1536595927903/Visualization-of-nucleation-at-the-interface-hypothesis-Modified-from-Long-1994-A.ppm)](https://www.researchgate.net/figure/Visualization-of-nucleation-at-the-interface-hypothesis-Modified-from-Long-1994-A_fig6_237824099)

(https://www.researchgate.net/figure/Visualization-of-nucleation-at-the-interface-hypothesis-Modified-from-Long-1994-A_fig6_237824099)

#### Local Structuring Nucleation Hypothesis

Recent molecular simulations have investigated the nucleation of gas hydrates in bulk water phase as well as the water-hydrocarbon interface, support the local structuring hypothesis over the labile cluster model (https://doi.org/10.1201/9781420008494). Radhakrishnan and Trout (2002) used Landau free energy calculations to study carbon dioxide hydrate nucleation. Their findings showed that labile clusters are more likely to disintegrate than to agglomerate, suggesting that nucleation occurs via local structuring (https://doi.org/10.1201/9781420008494). This was characterized by thermal fluctuations followed by the formation of a critical nucleus. Guest molecules (such as CO<sub>2</sub>) arrange in configurations similar to the clathrate hydrate phase, causing local perturbations in the bulk water phase, both due to stochastic thermodynamic fluctuations (based on system temperature). Eventually, the number of guest molecules in an ordered arrangement exceeds that in the critical nucleus, leading to hydrate formation, characterized by guest-guest and host-guest order parameters near the hydrate phase.



### <b> Types of Gas Hydrate Cage Recognition Algorithms</b>

Though clathrate hydrates have a vast array of applications, ranging from being a natural energy resource to CO<sub>2</sub> sequestration medium, several fundamental questions surrounding the formation, deformation and evolution of these structures remain unanswered. Due to both the scale and timescales associated with these structures, the only suitable method of investigation into their mechanisms is molecular dynamics (MD) simulations. Within these investigations, cage recognition serves as a measure of formation, and thus nucleation, as well as stability, in relation to thermodynamics. Many groups have proposed and published varying recognition algorithms with varying working principals and use cases. The most prevalent among these are: FISCA, CHILL+, GRADE, Recognition by Guest, ICO and Cup Recognition Algorithm.

#### FSICA Algorithm

The FISCA algorithm (and the variations thereof) work on the principal that a perfect polyhedron has edge and face saturation: each vertex joins at least three edges, and each edge connects two faces. Complete cages (CC) meet these conditions, while incomplete cages (IC) do not. Preceding ICA methods required existing methane molecules to identify surrounding cages. To overcome this limitation of ICA algorithms, Guo et al. devised the face-saturated incomplete cage analysis (FSICA) algorithm to identify complete cages (CC) and face-saturated incomplete cages (FSIC). In their seminal paper, FISCA was applied to water systems with H-bond networks in which there are both bonds and voids. Conceptually, the FISCA algorithm works by scanning entire H-bond network using a 3D mesh to find face-saturated cages. FSICA accurately identifies structures such as clusters of multiple hydrates and multi-guest molecular hydrates, which other algorithms can't achieve. While this algorithm can be used for both nucleation and core identification, it requires significant computing resources as well as a deep understanding of the target cage structure from implementation.
(https://doi.org/10.1016/j.jcrysgro.2022.126897)

![alt text](Zheng_et_al_Fig_2D_(FISCA).png "Title")

Adapted from: https://doi.org/10.1016/j.jcrysgro.2022.126897

#### CHILL+ Algorithm

Nguyen and Molinero developed the CHILL+ algorithm to identify cubic ice, hexagonal ice, clathrate hydrates, and liquids by analyzing staggered and overlapping water bonds. Unlike the original CHILL algorithm, which could only identify ice, CHILL+ was intedend as an extension to detect clathrate hydrates as well. The 3 categories of ice structure CHILL+ can identify all work based on patters in the number of staggered and eclipsed H-bonds. In cubic ice, each molecule forms four staggered bonds, while in hexagonal ice, three staggered bonds and one eclipsed bond are formed. Clathrate hydrates involve four eclipsed bonds per water molecule. Using these distinctions, CHILL+ can differentiate between ice and ice-like structures effectively though both the number of cages and their distribution in the system is required.
(https://doi.org/10.1016/j.jcrysgro.2022.126897)

![alt text](Zheng_et_al_Fig_2C_(CHILL+).png "Title")

Adapted from: https://doi.org/10.1016/j.jcrysgro.2022.126897

#### GRADE Algorithm

Using Matsumoto's code, Jacobson et al. identified 5- and 6-membered water rings using a depth-first search (DFS) where water molecules are analogous vertices of an undirected graph. This ring identification is subject to several adaptable parameters dictating the convexity and planarity of the identified rings. Then the connectivity of each identified ring are to other nearby rings is determined. Using the established connections, a ring (5 or 6 membered) is selected as the base before other 5 membered rings connected to each of its edges are selected, constructing a so-called "cup". These cups are essentially halves of cages, referred to as 5<sup>6</sup> 6<sup>1</sup>5<sup>6</sup> cups. By merging cups, complete hydrate cages, like 5<sup>12</sup>, 6<sup>2</sup>5<sup>12</sup> and 6<sup>4</sup>5<sup>12</sup> cages, can be identified. Mahmoudinobar and Dias extended the ring identification proposed by Matsumoto and Jacobson to create GRADE, which can identify 5<sup>12</sup>, 6<sup>2</sup>5<sup>12</sup> and 6<sup>4</sup>5<sup>12</sup> cage structures using combinations of two 5<sup>6</sup>, two 6<sup>1</sup>5<sup>6</sup> or four 6<sup>1</sup>5<sup>6</sup> cups to form each cage respectively. A major advantage of the GRADE algorithm is that it's code is open source, however, it also benefits from providing more cage formation information due to its hierarchical approach as well as F4 parameter calculation functionality. However, GRADE's functionality was limited to the cages studied, thus limiting its recognition capacity.
(https://doi.org/10.1016/j.jcrysgro.2022.126897)


#### Recognition of Cages by Guest Molecules

Lauricella et al. proposed a method to identify cages by searching for water molecules centered on guest molecules, subsequently avoiding the identification of empty cages. The group focused on only sI and sII types, identifying 5<sup>12</sup>, 5<sup>12</sup>6<sup>2</sup>, and 5<sup>12</sup>6<sup>4</sup> cages by finding 5- and 6-membered rings with Matsumoto's method. Then a modified Jacobson algorithm was used to merge these rings to form complete hydrate cages. As this approach uses guest molecules as a starting point, it effectively avoids empty cage defects.
(https://doi.org/10.1016/j.jcrysgro.2022.126897)

#### ICO Algorithm

Again, using the underlying method proposed by Jacobsen et al., Hao et al. proposed the Iterative Cup Overlap (ICO) algorithm to identify face-saturated cages by splicing cups. Each ring forms a cup with its adjacent ring, and by iteratively constructing cups topologically, complete cages are identified. This is done by using a ring as the base of a cup, before finding all the rings sharing an edge with this ring, referred to as "primary rings". Then each of the primary rings are used as a base ring and the same primary ring identification is carried out. The sets of primary rings between the first base and the primary rings used as bases are compared and the rings not common to both are stored as secondary rings. This sequential procedure essentially constructs the cup from only one starting cup rather than considering combinations of cups. Accordingly, this method can identify many cage types beyond 5<sup>12</sup>, 5<sup>12</sup>6<sup>2</sup>, and 5<sup>12</sup>6<sup>4</sup> with some adaptation.
(https://doi.org/10.1016/j.jcrysgro.2022.126897)

![alt text](Zheng_et_al_Fig_2B_(ICO).png "Title")

Adapted from: https://doi.org/10.1016/j.jcrysgro.2022.126897

### HTR Algorithm

Liu et al. proposed the HTR algorithm to identify hydrate cages using ring topology, again improving on the cup structure method by Jacobson et al. The HTR algorithm begins with identifying a base ring, an upper and a lower cup which share the ring before uniquely localizing secondary rings to complete the cages using two hydrogen bonds from three adjacent water molecules (see figure below). Notably, this avoids repetitive identification and combination of cups resulting in the highest efficiency in identifying multiple cage types developed thus far.

For complex hydrate structures where two rings can't form a unique topology, HTR performs topology calculations with one ring and then switches to another if needed. This ensures complete cage identification and prevents the reuse of rings already in a cage. HTR is highly efficient, especially for large-scale systems, as it introduces system splitting, reducing identification time from exponential to linear, making it suitable for studying hydrate micro-nano mechanics.
(https://doi.org/10.1016/j.jcrysgro.2022.126897)

![alt text](Zheng_et_al_Fig_2E_(HRT).png "Title")

Adapted from: https://doi.org/10.1016/j.jcrysgro.2022.126897

## <b><u> Objective </u></b>

The objective of the project was to create an easy-to-use implementation of the GRADE clathrate cage recognition algorithm. This new implementation aimed to be a more robust Python algorithm compared to the original C++ source code by Mahmoudinobar and Dias. The code should be easily extendable to different cage geometries and capable of recognizing various geometries, such as 5<sup>12</sup>, 6<sup>2</sup>5<sup>12</sup>, and 6<sup>4</sup>5<sup>12</sup> cages, using only .xyz file data in any order. The choice of .xyz file format was due to its widespread use in molecular modeling software, unlike the niche GROMAS (.gro) format used in the original paper. Additionally, the algorithm was designed to function as a Python module, utilizing an object-oriented approach to describe the network of water molecules as a graph object with structures as subclasses. Finally, it aimed to produce an output .xyz file of the identified cages for visualization.

## <b><u> Methods </u></b>

### Overview
The overall approach to cage identification is similar to the method highlighted in the original GRADE paper though the implementation differs relatively significantly compared to the open source code provided by Mahmoudinobar and Dias. Namely, the code provided by the group is written in C++ and does not use an object-oriented approach, rather storing all cage recognition results in a series of floats, integers and vectors initialized toward the beginning of the code. The approach proposed below uses a single graph class, encapsulating all the search functions, result variables and other information in the class. This design choice was made to facilitate the implementation of depth first search and to allow for malleable searches to be carried out (e.g. only 5<sup>12</sup> cages or only 6<sup>2</sup>5<sup>12</sup> cages or both). Additionally, the method by which rings, cups and cages were all adapted to more efficient (where possible), more general and thus work with unordered .xyz files.

The general use case of the GRADE implementation below runs as follows:

<b><u>Pre Processing:</u></b>

1. Read the .xyz file
2. Separate the various trajectory time steps if necessary
3. Store the .xyz file as a data frame using pd.read_csv() 
4. Segregate the Oxygen atoms from the Hydrogen atoms into two different data frames.

<u><b> Initialization: </b></u>

1. Make a Graph object and pass the Oxygen data frame as the argument
2. In the constructor, the list of Oxygen atom data is used to create a Vertex subclass object for each atom before populating the vertices list.
3. For each of the Vertex object created, the populate_neighbours() method is called to find the Oxygen atoms within a toleranced H-bond distance of the atom. These are stored in the neighbours set of the Vertex object.
4. Other key variables used later are initialized including: a boolean closeable variable, two lists for the rings identified, the identified cups and the identified cages and several temporary placeholder variables.

Once the graph has been initialized, the find_all() graph method can be called to run every identification procedure built into the implementation of GRADE.

<b><u>Execution:</u></b>

Ring Finding:

The ring finding procedure employs a depth first search algorithm of an undirected graph. Though unlike a traditional search algorithm which looks for node of a specific value, this search algorithm looks for a known pattern of nodes.

1. The fiveDFSRingFinder(i, 0) Graph method can be called to start the ring finding. The i is for the index of the starting Vertex in the vertices list initialized earlier, with 0 as the search depth.
2. The depth first search procedure is thus started, and will populate the temporary ring variable initialized earlier. This procedure is explained in detail later.
3. The add_ring_to_matrix() method will then populate the permanent ring list (five_ring_list / six_ring_list) with the rings stored in the temporary variable.
4. Each of the vertices is cleared of their search variables (visited and predecessor) using the clear() graph method.
5. Steps 1-4 are repeated starting at each vertex for both the 5 and 6 membered ring depth first search procedures, populating the five and six membered ring lists.

Neighbouring Rings:

Before any cups can be found, the neighbouring rings must be identified. This is done for both the 6-membered rings neighbouring 5-membered rings and the 5-membered rings neighbouring other 5-membered rings. This is all the information required for all cups topologies except type H cups (not covered in this poject).

6. Call both the five_find_neighbouring_five_rings() and the six_find_neighbouring_five_rings() methods. This iterates over the rings of a respective list and makes pairwise comparisons to rings of another list.
7. Each of the vertices of the pairs of rings are compared consecutively (ring vertices are stored in order) and if 2 molecules are common and consecutive in both rings, these rings are considered neighbours and the set variable, neighbouring_rings is updated as well as the coordination number.

Cup Finding:

Cup finding leverages the ring neighbour information found previously, and an iterative depth first search procedure to find 6<sup>1</sup>5<sup>6</sup> and 5<sup>6</sup> cups.

8. Call the find_6_1_5_6_cups() or find_5_6_cups() graph methods. For each either 5- or 6-membered ring identified, loop through all the remaining 5-membered rings, using them as the starting node for the depth-first search. This search uses rings rather than nodes (explained in detail later).
9. In the DFS procedure, checks are carried out at different search depths to find cups including duplicate cup, orientation and consistent neighbour checks. 
10. If a valid cup is found, it is added to either the cup_6_1_5_6_list or the cup_5_6_list before each of the rings is cleared of their search variables (visited and predecessor) using the clear() ring method.

Cage Finding:

Cage finding works by comparing pairs of identified cups using several geometric constraints.

11. The cage finding begins by looping over pairs of either 6<sup>1</sup>5<sup>6</sup> cups or 5<sup>6</sup> cups and checking they are oriented such that the normal vector of their bases point in opposing directions.
12. Then the distance between both base rings are compared to a set distance with a tolerance.
13. Finally, if the previous two checks are passed, the number of shared neighbour rings are counted to confirm the two cups form a cage.


### Ring Search Approach

The ring identification algorithm is takes a broadly similar approach to that of GRADE, though is different in some areas. Like GRADE, the DFS method checks for convexity and planarity, however, unlike GRADE, this implementation in recursive, object-oriented and perhaps more general than the open source code available. The key aspects of this crucial aspect of the entire cage identification method is highlighted in fiveDFSRingFinding() method. The only difference between the 5- and 6-membered ring finding methods are the convexity distance ratio, the interior angle value and the search depth at which the checks are carried out.

The entire method:

```python
## Recurvise depth-first search for PENTAGONAL rings
    def fiveDFSRingFinder(self, start_index, itter):
        # Get the vertex from the list of all verticies in the graph
        vertex = self.vertices[start_index]
        vertex.visited = True

        # Check if the current sequence of 3 points in already in a ring, if so, don't continue
        if itter >= 2:
            for ring in self.five_ring_list:
                for i in range(5):
                    if i == 3: # Edge case
                        if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.verticies[i], ring.verticies[i+1], ring.verticies[0]}:
                            return
                    elif i == 4: # Edge case
                        if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.verticies[i], ring.verticies[0], ring.verticies[1]}:
                            return
                    else:
                        if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.verticies[i], ring.verticies[i+1], ring.verticies[i+2]}:
                            return
            
            # Check the angle between the vectors is close to 108 degrees
            vec_theta = self.calc_vec_angle(vertex)
            if abs(vec_theta - 108) > FIVE_ANGLE_TOL:
                return
       
        # Check the angle between the planes formed by sets of 3 points, is less than 45 degrees, else don't continue
        if itter >= 3:
            plane_theta = self.calc_plane_angle(vertex)
            if plane_theta > PLANE_TILT_CUTOFF:
                return

        # Check if the sequence of 4 points (formed on the 3rd itteration) is closeable via a 5th point
        if itter == 3:
            # If any of the starting point's neighbours are neighbours of the current vertex's 3rd predecessor, the ring is closed
            for starting_point_neighbour in vertex.predecessor.predecessor.predecessor.neighbours:
                if vertex.is_neighbour(starting_point_neighbour):
                    self.closeable = True
                    self.five_ring.append(starting_point_neighbour)
                    return
                
        if not self.closeable: # Only run if the ring has not been closed (to avoid uneccesary computation)
            # Loop through each neighbour of the current vertex 
            for neighbour in vertex.neighbours:
                # DFS
                if not neighbour.visited and itter < 4 and not self.closeable:
                    # The neighbour's predecessor is now the current vertex (used for backtracing)
                    neighbour.predecessor = vertex
                    # Check the second vertex in the sequence is 1.6 times the HBond distance from the starting point, R(i, i+2) = 1.6*a
                    if itter >= 2:
                        if abs(1.6*vertex.dist(neighbour) - neighbour.dist(neighbour.predecessor.predecessor)) < FIVE_RING_TOL:
                            self.fiveDFSRingFinder(neighbour.index, itter + 1) # Recursion
                            if self.closeable:
                                self.five_ring.append(neighbour) # Only add to the ring variable if it is closable            
                    else:
                        if neighbour.neighbours is not None: 
                            self.fiveDFSRingFinder(neighbour.index, itter + 1) # Recursion
                        if self.closeable:
                            self.five_ring.append(neighbour)
```

Breakdown:

On the first call of this function (i.e. when the search is started from a given node), the index of the vertex in the vertices list is passed as well as a depth of 0 (required). The method then retrieves the vertex information from the vertices list and marks the vertex as visited.
```python
vertex = self.vertices[start_index]
vertex.visited = True
```
Then the first series of checks is carried out at a search depth greater than 2 corresponding to the 3rd vertex and later (i.e. iter=2). Search is equivalent to the number of the vertex of the pentagon. For example, if the search depth is 2, we are currently visiting the 3rd node in a sequence that may be a pentagon. In the code blow, 2 checks are carried out: duplicates and interior angles. The duplicate check compares the sequence of 3 nodes that have been visited on the current search (using the predecessor variable) to all sequences of 3 nodes in each of the rings that have already been identified. As a sequence of 3 nodes uniquely identifies a ring, this check effectively avoids double counting rings as the search is carried out, rather than after. This also ensures more rings are found as previously visited paths are not revisited. Several edge cases are considered to account for the 'loop' nature of the list of vertices that describe a ring.

```python
for ring in self.five_ring_list:
    for i in range(5):
        if i == 3: # Edge case
            if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.verticies[i], ring.verticies[i+1], ring.verticies[0]}:
                return
        elif i == 4: # Edge case
            if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.verticies[i], ring.verticies[0], ring.verticies[1]}:
                return
        else:
            if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.verticies[i], ring.verticies[i+1], ring.verticies[i+2]}:
                return
```

The other check is conducted on the 3rd vertex is the interior angle check. Here, the current vertex is passed to the method calc_vec_angle() which calculates two vectors using the 2 predecessors of the passed vertex before using the cosine angle identity to find the angle between them. If this angle is within a certain tolerance compared to a set value (108° for 5-membered rings and 120° for 6-membered rings), the search continues, else return and backtrack.

The check:
```python
vec_theta = self.calc_vec_angle(vertex)
if abs(vec_theta - 108) > FIVE_ANGLE_TOL:
    return
```
The angle method:
```python 
def calc_vec_angle(self, vertex):
    P = np.array([vertex.predecessor.predecessor.x, vertex.predecessor.predecessor.y, vertex.predecessor.predecessor.z])
    Q = np.array([vertex.predecessor.x, vertex.predecessor.y, vertex.predecessor.z])    
    R = np.array([vertex.x, vertex.y, vertex.z])

    QP = P - Q
    norm_QP = np.linalg.norm(QP)
    QR = R - Q
    norm_QR = np.linalg.norm(QR)

    theta = np.degrees(np.arccos(np.dot(QP, QR) / (norm_QP * norm_QR)))
    return theta
```

The final check, also carried out at a search depths greater than 2, is the convexity check which is only executed when deciding which node to visit next. This check calculates the current vertex's predecessor (i) distance to the next vertex (i+2) and compares it to the calculated distance between the current vertex(i+1) and the predecessor(i). If i to i+2 is within some tolerance of 1.6 times i to i+1, select this node to continue the search. For hexagonal rings, i to i+3 is compared to 2 times i to i+1.

```python
if abs(1.6*vertex.dist(neighbour) - neighbour.dist(neighbour.predecessor.predecessor)) < FIVE_RING_TOL:
    self.fiveDFSRingFinder(neighbour.index, itter + 1) # Recursion
```

![alt text](Dias_et_al_Fig_3.png "Title")

Adapted from: https://doi.org/10.1016/j.cpc.2019.06.004

There is only 1 check carried out at a search depth of 3 or greater, the planarity check. In the DFS method, the current vertex is passed to the calc_plane_angle() method. This method constructs a plane using the 3rd, 2nd and 1st predecessor and another plane 3rd and 1st predecessors and the current index. Then the normal vectors of these planes are found and the angle between them compared to some set value (45°) with a tolerance to ensure the ring is planar.

The check :
```python
if itter >= 3:
    plane_theta = self.calc_plane_angle(vertex)
    if plane_theta > PLANE_TILT_CUTOFF:
        return
```

The method:
```python
def calc_plane_angle(self, vertex):
    P = np.array([vertex.predecessor.predecessor.predecessor.x, vertex.predecessor.predecessor.predecessor.y, vertex.predecessor.predecessor.predecessor.z])
    Q = np.array([vertex.predecessor.predecessor.x, vertex.predecessor.predecessor.y, vertex.predecessor.predecessor.z])    
    R = np.array([vertex.predecessor.x, vertex.predecessor.y, vertex.predecessor.z])
    S = np.array([vertex.x, vertex.y, vertex.z])

    PQ = Q - P
    PR = R - P
    normal_1 = np.cross(PQ, PR)
    norm_1 = np.linalg.norm(normal_1)

    PR = R - P
    PS = S - P
    normal_2 = np.cross(PR, PS)
    norm_2 = np.linalg.norm(normal_2)

    theta = np.degrees(np.arccos(np.dot(normal_1, normal_2) / (norm_1 * norm_2)))
    return theta
```

![alt text](Dias_et_al_Fig_4.png "Title")

Adapted from: https://doi.org/10.1016/j.cpc.2019.06.004


The final and key check is whether the ring is closeable and thus occurs at a search depth of exactly 3 (4th vertex). If a depth of 3 is reached, each of the previous checks have been passed and thus, to check the ring is closeable, a valid neighbour of the current vertex is compared to the set of neighbours of the vertex used as the starting point of the search. If the current vertex (4th vertex) and the starting vertex share a common neighbour, the ring is closeable and we have found a 5-membered ring.

NOTE: The 6-membered ring finding algorithm is the same only the convexity check and search depths are adapted based on the different hexagonal shape.

```python
for starting_point_neighbour in vertex.predecessor.predecessor.predecessor.neighbours:
    if vertex.is_neighbour(starting_point_neighbour):
        self.closeable = True
        self.five_ring.append(starting_point_neighbour)
        return
```

### Neighbouring Rings

In order to move from identifying rings (collections of Vertex objects) to cups (collections of Ring objects), the number and identitiy of a given ring's neighbours must be identified. Each Ring object, thus contains a set of neighbouring rings (also Ring objects), which will later allow for the cup recoginition method. 

Neighbouring rings are defined as two rings which share a pair of consecutive vertices (i.e. and edge of the hexagon/pentagon). The method to populate each identified ring's neighbours works simply by comparing each pair of consecutive vertices of one ring to another. This is done by looping through pairs of rings making each pair of vertices set objects and checking for equality as this avoids errors in the ordering of the compared vertices. However, Ring objects are set up to hold ordered lists of Vertex objects (in order of the vertices of a hexagon/pentagon), meaning there is both a 'beginning' and 'end', while a ring inherently has neither. Therefore, while populating the set of neighbours for a given ring, several cases considering the looping of a ring are checked.

```python 
    def six_find_neighbouring_five_rings(self):
        # For each hexagonal ring, find neighbouring pentagonal rings
        for hex_ring in self.six_ring_list:
            for pent_ring in self.five_ring_list:  
                for i in range(6): 
                    for j in range(5):
                        if i == 5 and j == 4: # Edge case (loop around from the last vertex to the first vertex, both rings)
                            if {hex_ring.vertices[i], hex_ring.vertices[0]} == {pent_ring.vertices[j], pent_ring.vertices[0]}: 
                                hex_ring.neighbouring_rings.add(pent_ring)
                                pent_ring.neighbouring_rings.add(hex_ring)
                                hex_ring.coordination += 1
                                pent_ring.coordination += 1
                        elif i == 5: # Edge case (loop around from the last vertex to the first vertex, hexagon)
                            if {hex_ring.vertices[i], hex_ring.vertices[0]} == {pent_ring.vertices[j], pent_ring.vertices[j+1]}: 
                                hex_ring.neighbouring_rings.add(pent_ring)
                                pent_ring.neighbouring_rings.add(hex_ring)
                                hex_ring.coordination += 1
                                pent_ring.coordination += 1
                        elif j == 4: # Edge case (loop around from the last vertex to the first vertex, pentagon)
                            if {hex_ring.vertices[i], hex_ring.vertices[i+1]} == {pent_ring.vertices[j], pent_ring.vertices[0]}: 
                                hex_ring.neighbouring_rings.add(pent_ring)
                                pent_ring.neighbouring_rings.add(hex_ring)
                                hex_ring.coordination += 1
                                pent_ring.coordination += 1
                        else:
                            if {hex_ring.vertices[i], hex_ring.vertices[i+1]} == {pent_ring.vertices[j], pent_ring.vertices[j+1]}: 
                                hex_ring.neighbouring_rings.add(pent_ring)
                                pent_ring.neighbouring_rings.add(hex_ring)
                                hex_ring.coordination += 1
                                pent_ring.coordination += 1
```
NOTE: There are 2 methods to populate the neighbours of different rings which make up cups, one for the 6-membered rings neighbouring 5-membered rings and another for 5-membered rings neighbouring other 5-membered rings. This was done as 6<sup>2</sup>5<sup>12<sup> cups are made up of 6-membered rings neighbouring 5-membered rings while 5<sup>12</sup> are made up of 5-membered rings neighbouring other 5-membered rings. Subsequently, only one of the methods could be called to find each rings neighbours depending on the cups of interest. Both methods are very similar and could be refactored into a single method


### Cup Search Approach

The cup identification approach takes inspiration from the ring finding approach though, rather than using Vertex objects to conduct the DFS undirected graph search, rings are used instead. Essentially, we start with a base ring, then perform DFS to find a loop in the base ring's neighbouring rings subject to the constraint that each ring is oriented in the same direction. Accordingly, each Ring object contains the same predecessor and visited variables. Additionally, the DFS algorithm for the ring search is iterative rather than recursive and is set up to identify several cups from a single starting ring as this is common is hydrate structures. However, similar to the ring recognition DFS, the cup search has several checks at different search depths, essentially corresponding to the rings sharing an edge with the base ring which make up the cup. Again, rather than looking for a given node, we are looking for a pattern of nodes.

The entire method:

```python
def find_6_1_5_6_cups(self):
    for hex_ring in self.six_ring_list:
        if hex_ring.coordination >= 6: # If the hexagon is fully coordinated
            hex_ring.visited = True
            for neighbouring_ring in hex_ring.neighbouring_rings: # For each neighbouring pentagon
                stack = [(neighbouring_ring, 0)] # Add the pentagon to the stack with a depth of 0
                skip = False
                # Itterative depth first search
                while stack: 
                    current_ring, depth = stack.pop()
                    current_ring.visited = True
                    if not skip:
                        # Check if any sequence of 3 rings (depth 2) is common to an already found cup
                        if depth >= 2 and self.cup_6_1_5_6_list:
                            for cup_6_1_5_6 in self.cup_6_1_5_6_list:
                                for i in range(1, len(cup_6_1_5_6.rings)):
                                    if i == 5: # Edge case
                                        if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_6_1_5_6.rings[i], cup_6_1_5_6.rings[i+1], cup_6_1_5_6.rings[1]}:
                                            skip = True
                                            break 
                                    elif i == 6: # Edge case
                                        if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_6_1_5_6.rings[i], cup_6_1_5_6.rings[1], cup_6_1_5_6.rings[2]}:
                                            skip = True
                                            break
                                    else:
                                        if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_6_1_5_6.rings[i], cup_6_1_5_6.rings[i+1], cup_6_1_5_6.rings[i+2]}:
                                            skip = True # Don't continue, we have already found this cup
                                            break
                                if skip: # Break out of the exterior loop
                                    break
                        
                        if depth > 4: # At a depth of 5, we are on the 6th pentagon
                            if current_ring.predecessor.predecessor.predecessor.predecessor.predecessor in current_ring.neighbouring_rings:# Check if the 5th ring is the root, then we have a 6_1_5_12 cup
                                self.cup_6_1_5_6_list.append(self.Cup([hex_ring,
                                                                        current_ring.predecessor.predecessor.predecessor.predecessor.predecessor, 
                                                                        current_ring.predecessor.predecessor.predecessor.predecessor, 
                                                                        current_ring.predecessor.predecessor.predecessor, 
                                                                        current_ring.predecessor.predecessor, 
                                                                        current_ring.predecessor,
                                                                        current_ring])) # Add the cup to the list
                                break
                                
                        for ring in current_ring.neighbouring_rings: # DFS
                            if not ring.visited and ring.is_neighbour_of(hex_ring): # Ring needs to be a neighbour of the hexagon
                                if current_ring.predecessor is not None:
                                    if ring.neighbour_orientation(hex_ring) == current_ring.predecessor.neighbour_orientation(hex_ring): # Ring needs to be on the same side of the hexagon as the previous ring
                                        ring.predecessor = current_ring
                                        stack.append((ring, depth + 1)) # Add the ring to the stack with a depth of 1 greater than the current ring
                                else:
                                    ring.predecessor = current_ring
                                    stack.append((ring, depth + 1))
                # Clear all visited flags
                for ring in self.five_ring_list:    
                    ring.clear()
```

Breakdown:

When the find_6_1_5_6_cups() method (or find_5_6_cups()) is called, the method populates the cup_6_1_5_6_list (or cups_5_6_list), thus is it assumed that either both of the DFSRingFinding() methods have been called prior.

First, the method iterates over all hexagonal rings identified earlier and checks if the current 6-memebered ring has a coordination of at least 6, thus indicating it has enough neighbouring rings to form a cup. If the previous condition is met, the neighbours of the 'base' ring of the cup are iterated over, performing a DFS on each neighbour. As the DFS is iterative, a stack is initalized with the current neighbour as the first element of the stack. A while loop signifies the beginning of the DFS.

```python
for hex_ring in self.six_ring_list:
        if hex_ring.coordination >= 6: # If the hexagon is fully coordinated
            hex_ring.visited = True
            for neighbouring_ring in hex_ring.neighbouring_rings: # For each neighbouring pentagon
                stack = [(neighbouring_ring, 0)] # Add the pentagon to the stack with a depth of 0
                skip = False
                # Itterative depth first search
                while stack: 
                    current_ring, depth = stack.pop()
                    current_ring.visited = True

                    ...
```

As with the ring finding approach, the geometric nature of the hydrate cage necessitates several checks at different depths. The earliest of these is almost identical to the duplicate check in the ring finding. At a search depth of 2 (3rd consecutive ring), if there is a cup which has already been identified (i.e., in cup_6_1_5_6_list), ignore this node by setting the skip flag to true. This check ensures no duplicate cups are found and avoids the need to filter cups out of the searhc

```python
...
if depth >= 2 and self.cup_6_1_5_6_list:
    for cup_6_1_5_6 in self.cup_6_1_5_6_list:
        for i in range(1, len(cup_6_1_5_6.rings)):
            if i == 5: # Edge case
                if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_6_1_5_6.rings[i], cup_6_1_5_6.rings[i+1], cup_6_1_5_6.rings[1]}:
                    skip = True
                    break 
            elif i == 6: # Edge case
                if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_6_1_5_6.rings[i], cup_6_1_5_6.rings[1], cup_6_1_5_6.rings[2]}:
                    skip = True
                    break
            else:
                if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_6_1_5_6.rings[i], cup_6_1_5_6.rings[i+1], cup_6_1_5_6.rings[i+2]}:
                    skip = True # Don't continue, we have already found this cup
                    break
        if skip: # Break out of the exterior loop
            break
...
```

The next check occurs at search depth of 5, meaning we are on the 6th pentagon which may close the loop of pentagons bordering a base hexagon, corresponding to a cup structure. Thus, at this depth, the method compares the current ring to the neighbours of the starting neighbour of the base ring. If said current ring is a neighbour of the starting ring, we have found a cup, so append it to the list of identified cups. However, if a cup is found, the ring-based DFS simply continues to a new starting neighbour of the base ring after appending, meaning cups formed on either side of the same base ring.

```python
if depth > 4: # At a depth of 5, we are on the 6th pentagon
    if current_ring.predecessor.predecessor.predecessor.predecessor.predecessor in current_ring.neighbouring_rings:# Check if the 5th ring is the root, then we have a 6_1_5_12 cup
        self.cup_6_1_5_6_list.append(self.Cup([hex_ring,
                                                current_ring.predecessor.predecessor.predecessor.predecessor.predecessor, 
                                                current_ring.predecessor.predecessor.predecessor.predecessor, 
                                                current_ring.predecessor.predecessor.predecessor, 
                                                current_ring.predecessor.predecessor, 
                                                current_ring.predecessor,
                                                current_ring])) # Add the cup to the list
        break

    ... # Continue DFS
```

The final check combined is carried out when assesing the next ring the DFS algortihm can move on too. The two conditions are that the next ring candidate both neighbours the base ring and is oriented in the same direction (angled with respect to the base ring) as the current ring in the sequence of rings making up the cup. This check is carried out for all rings following the starting base neighbour ring. 


The orientation of the ring is determined by the Ring function, neighbour_orientation(), which takes in the base ring with which to calculate the orientation with respect to. The method calculates either the sign or the value of the normal of the base neighbouring ring, depending on the input mode. This is achecived using dot product of the the normal vector of the base as well as a vertex of the base neighbouring ring which is not common to the base ring. See the first stack exchange link for further details.

https://math.stackexchange.com/questions/2998875/how-to-determine-if-a-point-is-above-or-below-a-plane-defined-by-a-triangle

The check:

```python 
if not ring.visited and ring.is_neighbour_of(hex_ring): # Ring needs to be a neighbour of the hexagon
    if current_ring.predecessor is not None:
        if ring.neighbour_orientation(hex_ring) == current_ring.predecessor.neighbour_orientation(hex_ring): # Ring needs to be on the same side of the hexagon as the previous ring
            ring.predecessor = current_ring
            stack.append((ring, depth + 1)) # Add the ring to the stack with a depth of 1 greater than the current ring
    else:
        ring.predecessor = current_ring
        stack.append((ring, depth + 1))
```

The method:

![alt text](Math_Exchange_img.png "Title")

(Depiction of the idea behind the orientation method)
https://math.stackexchange.com/questions/4409996/how-to-check-if-a-point-is-behind-a-plane-along-a-vector 

```python
def neighbour_orientation(self, base_ring, return_oriented_normal=False):
    P = np.array([base_ring.vertices[1].x, base_ring.vertices[1].y, base_ring.vertices[1].z])
    Q = np.array([base_ring.vertices[2].x, base_ring.vertices[2].y, base_ring.vertices[2].z])
    R = np.array([base_ring.vertices[0].x, base_ring.vertices[0].y, base_ring.vertices[0].z])
    hex_vec_1 = Q - P
    hex_vec_2 = R - P
    hex_normal = np.cross(hex_vec_1, hex_vec_2)

    for vertex in self.vertices:
        if vertex not in base_ring.vertices:
            N = np.array([vertex.x, vertex.y, vertex.z])
            if return_oriented_normal:
                return (hex_normal * np.sign(np.dot(hex_normal, N - P)))/np.linalg.norm(hex_normal)
            return np.sign(np.dot(hex_normal, N - P))
```

NOTE: The 5<sup>6</sup> cup finding algorithm is the same only the types rings compared in pairs and search depths are adapted based on the different shapes.

### Cage Finding Approach

The cage finding method is one which is purely geometric and leverages the idea that a cage is formed by two cups sharing a constant number of edges between them. The geometric aspects of this function relate to the orientation between cups (i.e., that they point in toward one another), and the distance separating their base centroids. These geometric checks help filter out invalid pairs of cups before the vertices of the rings of a cup are iterated through. As has been the theme with the entire algorithm thus far, this step is hierarchical meaning the function's ability to identify cages depends on the success of the cup identification method, in turn depending on ring finding. Thus, when tweaking algorithm constants, one should first adapt those pertaining to rings, then cups, then cages, to essentially calibrate the algorithm. Additionally, as is perhaps evident, both the ring finding and cup finding functions must be called before this one.

The entire method:

```python 
def find_6_2_5_12_cages(self):
    for i in range(len(self.cup_6_1_5_6_list) - 1):
        cup_1 = self.cup_6_1_5_6_list[i]
        for j in range(i, len(self.cup_6_1_5_6_list)):
            common_edge_count = 0
            cup_2 = self.cup_6_1_5_6_list[j]
            if np.isclose(np.dot(cup_1.oriented_normal, (-1)*cup_2.oriented_normal), 1, rtol=CAGE_6_2_5_12_BASE_TILT_TOL) and cup_1.base != cup_2.base:
                if abs(cup_1.base_centroid().dist(cup_2.base_centroid()) - CAGE_6_2_5_12_HEIGHT) < CAGE_6_2_5_12_HEIGHT_TOL:
                    for k in range(1, len(cup_1.rings)):
                        for j in range(1, len(cup_2.rings)):
                            if cup_1.rings[k].is_neighbour_of(cup_2.rings[j]):
                                common_edge_count += 1
                    if common_edge_count == 12:
                        self.cage_6_2_5_12_list.append(self.Cage([cup_1, cup_2])) 
``` 

The breakdown:

The method find_6_2_5_12_cages() is a method within the Graph class and takes no inputs as it accesses the already populated cup lists. Each pair of a certain type of cup is iterated over depending on the cage to be identified (e.g., 6<sup>1</sup>5<sup>6</sup> cups for 6<sup>2</sup>5<sup>12</sup> cages or 5<sup>6</sup> cups for 5<sup>12</sup> cages). This is done using two for loops and pairwise indexing. For each pair, a temporary variable storing the number of edges common to both cups, common_edge_count, is initialized.

```python
for i in range(len(self.cup_6_1_5_6_list) - 1):
    cup_1 = self.cup_6_1_5_6_list[i]
    for j in range(i, len(self.cup_6_1_5_6_list)):
        common_edge_count = 0
    
    ...
```

Once a pair has been accessed from a given cup list, the orientation of the two cups is calculated and compared to ensure the cups are facing one another, the possibly forming a cage. This is done using the dot product between the oriented normal vectors of the pair of cups is taken. If the result is near 1 (within a tunable tolerance), the vectors are oriented toward one another. 

```python
if np.isclose(np.dot(cup_1.oriented_normal, (-1)*cup_2.oriented_normal), 1, rtol=CAGE_6_2_5_12_BASE_TILT_TOL) and cup_1.base != cup_2.base:
```

![alt text](Dias_et_al_Fig_6.png 'Title')

(Image depicting the orientation idea behind the cage method) (https://doi.org/10.1016/j.cpc.2019.06.004)

If the previous check is passed, the distance between both cups is calculated and compared to a cage height constant (the distance between cup base centroids), within a tolerance. This is done using the dist() Vertex method as the base_centroid() Cup method return a Vertex object representing the centroid of the base of the cup.

```python
if abs(cup_1.base_centroid().dist(cup_2.base_centroid()) - CAGE_6_2_5_12_HEIGHT) < CAGE_6_2_5_12_HEIGHT_TOL:
```

Finally, once both of these geometric constraints are met, the function iterates over each of the vertices of the rings which make up the pairs of cups, excluding the base rings, and counts the number of common edges between them. This is done in the same way as the neighbour finding methods. For a 6<sup>2</sup>5<sup>12</sup> cage, the cups which make it up will share 12 common edges, while 5<sup>12</sup> cages share 10. If this constraint is met, a cage has been rigorously identified, a Cage object is added to the list of cages.

```python
for k in range(1, len(cup_1.rings)):
        for j in range(1, len(cup_2.rings)):
            if cup_1.rings[k].is_neighbour_of(cup_2.rings[j]):
                common_edge_count += 1
    if common_edge_count == 12:
        self.cage_6_2_5_12_list.append(self.Cage([cup_1, cup_2])) 
```

NOTE: The 6<sup>2</sup>5<sup>12</sup> and the 5<sup>12</sup> cage methods are equivalent only differing in their constants and the number of common edges.



## <b><u> Code Walkthrough </b></u>

### Imports

The only methods required are those which are relatively standard for data science / scientific computing. These are pandas for handling the .xyz data, numpy for all the linear algebra, matplotlib for plotting (debugging mainly) and itertools is intended to be used for 6<sup>4</sup>5<sup>12</sup> cups, though this method could not be efficiently developed and so is excluded (>12 min execution time for just that method).

In [1]:
%matplotlib widget
# other options for matplotlib are: `widget`, `ipympl`, `notebook`
# default is `widget`

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

## Accesing the .xyz File 

<u>THIS IS WHAT YOU SHOULD CHANGE FOR YOUR USE <b>(edit the filename)</b></u>. Here, the .xyz chemical files are read and a dataframe of all the atoms included is created. Then, the with operator and open are used to access the file in read mode. The number of atoms is the accessed as this integer is always the first line of an .xyz file. After this, the dataframe is made using pandas and the read_csv() function using the parameters read earlier to skip the first two lines, then read all the lines after line 2 for the number of atoms. The column names are adpated.

In [4]:
# trajectory/si_interface_waters_cut_frame.10.xyz
# trajectory/si_interface_waters.xyz
# trajectory/double_interface_waters.xyz
# sii_empty.xyz
# 5-12_OH.xyz
#file_name = '/Users/nicksmits/chee494/reference_files/trajectory/si_interface_waters.xyz'
file_name = '../reference_files/trajectory/si_interface_waters.xyz'

# Open the .xyz file to read the first line to get the number of atoms
with open(file_name, 'r') as file:
    num_atoms = int(file.readline().strip())  # Number of atoms
    comment_line = next(file)  # Skip the comment line

# Now, read the atomic data into a DataFrame
# Assuming the atomic data starts immediately after the first two lines
# and consists of columns like: atom type, x-coordinate, y-coordinate, z-coordinate
df = pd.read_csv(file_name, 
                 delim_whitespace=True, 
                 skiprows= 2,  # Skip the first two lines
                 nrows=num_atoms,  # Read only the specified number of atoms
                 names=['atom', 'x', 'y', 'z'])  # Adjust column names as necessary

/tmp/ipykernel_160525/1191205851.py:17: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_name,


In [5]:
#column_names = ['atom', 'x', 'y', 'z']
## sI_Reference.xyz
## si_co2/co2_rand_relaxed.xyz
## si_ch4/methane_relaxed.xyz
## trajectory/double_interface_waters.xyz
##file_name = '/Users/nicksmits/chee494/reference_files/trajectory/#double_interface_waters.xyz'
#file_name = '../reference_files/trajectory/si_interface_waters.xyz'
#df = pd.read_csv(file_name,
#                 sep='\s+',
#                 names=column_names,
#                 skiprows=2)
## the `sep='\s+` is necessary because I want to use whitespace
## as the separator and not just `sep=" "`

ParserError: Error tokenizing data. C error: Expected 4 fields in line 3730, saw 13


In [ ]:
df

,atom,x,y,z
0,2,2.882721,-0.173053,4.460363
1,1,0.209483,0.865332,3.313437
2,2,0.783554,0.949310,8.472308
3,1,0.045218,2.959617,1.472248
4,1,6.115198,3.873719,0.083113
...,...,...,...,...
3721,2,28.348190,31.926849,30.755176
3722,1,31.370114,31.485852,30.138362
3723,2,31.948095,30.723988,30.097977
3724,2,31.618846,32.016216,29.381302


### Other Preprocessing

To speed up execution, only Oxygen atoms are used to identify the cages and thus the dataframe, df, is split into the hydrogen and the oxygen atoms. Depending on the .xyz file, the atoms types may be labelled using letters (e.g. 'O' or 'H') or using numbers (e.g. 1 or 2). Uncomment the lines below accordingly.

<u>CHECK YOUR FILE AND CHANGE ACCORDINGLY</u>

In [ ]:
## Separate .xyz file into oxygen and hydrogen atom dataframes ##

# oxygen_df = df[df['atom'] == 'O']
# hydrogen_df = df[df['atom'] == 'H']

oxygen_df = df[df['atom'] ==  1]
hydrogen_df = df[df['atom'] == 2]

# oxygen_df = oxygen_df.sample(frac=1)

oxygen_df

,atom,x,y,z
1,1,0.209483,0.865332,3.313437
3,1,0.045218,2.959617,1.472248
4,1,6.115198,3.873719,0.083113
5,1,6.192214,2.321250,2.376905
9,1,3.658912,7.885912,0.094353
...,...,...,...,...
3709,1,33.130079,29.380004,29.998910
3712,1,34.592373,29.235400,32.335232
3715,1,34.751891,29.309274,27.750395
3719,1,28.599484,31.408723,29.990537


In [ ]:
# Plot 3D atoms point clouds

# fig = plt.figure(figsize=plt.figaspect(0.5))
# ax = fig.add_subplot(1, 2, 1, projection='3d')

# # Plot all atoms
# ax.scatter(df['x'], df['y'], df['z'],
#            marker='o',
#            s=100)
# ax = fig.add_subplot(1, 2, 2, projection='3d')

# # Plot oxgygens only 
# ax.scatter(oxygen_df['x'], oxygen_df['y'], oxygen_df['z'],
#             marker='o',
#             s=100, 
#             c='r')
# plt.show()

## Other Methods

Below are 3 methods which may aid in determining some constants for the cage finding algorithm. These include dist which calculates the distance between 2 atoms from rows of the dataframes defined above, inteded to help determine HBOND_DIST, HBOND_DIST_TOL, FIVE_RING_TOL and SIX_RING_TOL. the method angle_vec() is used to calculate the angle between a set of 3 vertices, using the central atom as the common point for the 2 vectors used to find the angle. This is useful when determining FIVE_ANGLE_TOL and SIX_ANGLE_TOL. The final method, angle_plane() finds the angle between sets of 4 vertices by determining planes from sets of 3 vertices (see the ring finding approach above for more details). This is useful when calculating PLANE_TILT_CUTOFF.

Use these methods with the commented out cells below if the already calibrated constants fail to identify cages correctly.

In [ ]:
def dist(P, Q):
    return np.sqrt((P[0]-Q[0])**2 + (P[1]-Q[1])**2 + (P[2]-Q[2])**2)

In [ ]:
def angle_vec(P, Q, R):
    P = np.array(P)
    Q = np.array(Q)
    R = np.array(R)

    PQ = Q - P
    QR = R - Q

    dot_product = np.dot(PQ, QR)
    norm_PQ = np.linalg.norm(PQ)
    norm_QR = np.linalg.norm(QR)

    return np.degrees(np.arccos(dot_product / (norm_PQ * norm_QR)))

In [ ]:
def angle_plane(P, Q, R, S):
    P = np.array(P, dtype='float64')
    Q = np.array(Q, dtype='float64')
    R = np.array(R, dtype='float64')
    S = np.array(S, dtype='float64')

    PQ = Q - P
    PR = R - P
    PS = S - P

    normal_1 = np.cross(PQ, PR)
    norm_1 = np.linalg.norm(normal_1)

    normal_2 = np.cross(PR, PS)
    norm_2 = np.linalg.norm(normal_2)

    dot_product = np.dot(normal_1, normal_2)

    return np.degrees(np.arccos(dot_product / (norm_1 * norm_2)))

In [ ]:
# Known HBond atoms
# oxygen_df.iloc[20]
oxygen_df.iloc[3][:]

atom    1.000000
x       6.192214
y       2.321250
z       2.376905
Name: 5, dtype: float64

In [ ]:
# hydrogen_df.iloc[25][:]

In [ ]:
# print(dist(oxygen_df.iloc[17][1:], hydrogen_df.iloc[3][1:]))

In [ ]:
# # Ball park HBond / FIVE_RING_TOL / SIX_RING_TOL Dist value 
# Q_idx = 60 # --> Left Atom
# P_idx = 37 # --> Middle Atom
# R_idx = 34 # --> Right Atom
# S_idx = 56 # --> Right Right Atom

# # Distance between P and Q --> For HBOND_DIST
# dist(oxygen_df.iloc[S_idx][1:], oxygen_df.iloc[R_idx][1:])

# # Ratio 1->3/1->2 of distances --> For FIVE_RING_TOL 
# dist(oxygen_df.iloc[Q_idx][1:], oxygen_df.iloc[R_idx][1:]) / dist(oxygen_df.iloc[P_idx][1:], oxygen_df.iloc[Q_idx][1:])

# # Ratio 1->4/1->2 of distances --> For SIX_RING_TOL
# dist(oxygen_df.iloc[P_idx][1:], oxygen_df.iloc[S_idx][1:]) / dist(oxygen_df.iloc[P_idx][1:], oxygen_df.iloc[Q_idx][1:])

# # Angle between P, Q, R --> For FIVE_ANGLE_TOL or SIX_ANGLE_TOL
# angle_vec(oxygen_df.iloc[P_idx][1:], oxygen_df.iloc[Q_idx][1:], oxygen_df.iloc[R_idx][1:])

# # Angle between two planes P, Q, R and P, R, S --> For PLANE_CUTOFF
# angle_plane(oxygen_df.iloc[Q_idx][1:], oxygen_df.iloc[P_idx][1:], oxygen_df.iloc[R_idx][1:], oxygen_df.iloc[S_idx][1:])

## The Cage Finding Algorithm

Below is the cell containing the Graph method which is used to do all the identification described in detail in the methods section. Simply run the following cell to define it before using the algorithm.

<u>THE ONLY THING TO CHANGE BELOW ARE THE CONSTANTS IN CAPS</u>

In [ ]:
# Constants (They have been tuned to work with the data set, they may need to be adjusted for other data sets)
HBOND_DIST = 2.6
HBOND_DIST_TOL = 0.75 # 0.75

FIVE_RING_TOL = 0.6 # i to i+2 distance ratio tolerance
FIVE_ANGLE_TOL = 40 # Degrees (interior angle, this has to be large due to not perfectly planar rings)

SIX_RING_TOL = 0.6 # i to i+3 distance ratio tolerance
SIX_ANGLE_TOL = 40 # Degrees (interior angle, this has to be large due to not perfectly planar rings)

PLANE_TILT_CUTOFF = 45 # Degrees (45 in paper)

CAGE_6_2_5_12_HEIGHT = 6.015 # For 6_2_5_12 cages
CAGE_6_2_5_12_HEIGHT_TOL = 0.8 # For 6_2_5_12 cages
CAGE_6_2_5_12_BASE_TILT_TOL = 0.2 # For 6_2_5_12 cages

CAGE_6_4_5_12_ADJ_HEX_DIST = 6.148 # For 6_4_5_12 cages
CAGE_6_4_5_12_ADJ_HEX_DIST_TOL= 0.8 # For 6_4_5_12 cages

CAGE_5_12_HEIGHT = 6.5 # For 5_12 cages
CAGE_5_12_HEIGHT_TOL = 0.9 # For 5_12 cages
CAGE_5_12_BASE_TILT_TOL = 0.5 # For 5_12 cages

# Output Params
aspect_ratio = [1, 1, 1] # Aspect ratio for output graphs if using the plot functions 

class Graph:
    def __init__(self, vertices): # input a dataframe with 0 = index, 1,2,3 = x,y,z
        # Create all verticies and find neighbours for each vertex
        self.vertices = []
        for i in range(len(vertices)):
            self.vertices.append(self.Vertex(
                i, 
                vertices.iloc[i][1], 
                vertices.iloc[i][2],
                vertices.iloc[i][3]
            ))
        
        # Populate neighbours for each vertex
        for vertex in self.vertices:
            vertex.populate_neighbours(self.vertices)

        self.closeable = False # Closeable ring flag

        self.five_ring = [] # Temporary variable for 5 membered rings
        self.five_ring_list = [] # List of 5 membered rings

        self.six_ring = [] # Temporary variable for 6 membered rings
        self.six_ring_list = [] # List of 6 membered rings

        self.cup_6_1_5_6_list = [] # List of 6_1_5_12 cups
        self.cup_5_6_list = [] # List of 5_6 cups

        self.cage_6_2_5_12_list = [] # List of 6_2_5_12 cages
        self.cage_5_12_list = [] # List of 5_12 cages   
        # self.cage_6_4_5_12_list = [] # List of 6_4_5_12 cages

    # Function to find all rings, cups, and cages
    def find_all(self):
        for i in range(len(self.vertices)):
            self.fiveDFSRingFinder(i, 0)
            self.add_ring_to_matrix(i)
            self.clear()

        for i in range(len(self.vertices)):
            self.sixDFSRingFinder(i, 0)
            self.add_ring_to_matrix(i)
            self.clear()

        self.five_find_neighbouring_five_rings()
        self.six_find_neighbouring_five_rings()

        self.find_6_1_5_6_cups()
        self.find_5_6_cups()

        self.find_6_2_5_12_cages()
        self.find_5_12_cages()
        # self.find_6_4_5_12_cages()
    
    # Print a summary of the search 
    def summary(self):
        print("Summary:")
        print("Rings:")
        print("\t 5 membered rings: " + str(len(self.five_ring_list)))
        print("\t 6 membered rings: " + str(len(self.six_ring_list)))
        print("Cups:")
        print("\t 6_1_5_12 cups: " + str(len(self.cup_6_1_5_6_list)))
        print("\t 5_6 cups: " + str(len(self.cup_5_6_list)))
        print("Cages:")
        print("\t 6_2_5_12 cages: " + str(len(self.cage_6_2_5_12_list)))
        print("\t 5_12 cages: " + str(len(self.cage_5_12_list)))
        # print("\t 6_4_5_12 cages: " + str(len(self.cage_6_4_5_12_list)))

    # NOTE: Both of the following DFS methods could be combined into a single method with more inputs

    ## Recurvise depth-first search for PENTAGONAL rings
    def fiveDFSRingFinder(self, start_index, itter):
        # Get the vertex from the list of all verticies in the graph
        vertex = self.vertices[start_index]
        vertex.visited = True

        # Check if the current sequence of 3 points in already in a ring, if so, don't continue
        if itter >= 2:
            for ring in self.five_ring_list:
                for i in range(5):
                    if i == 3: # Edge case
                        if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.vertices[i], ring.vertices[i+1], ring.vertices[0]}:
                            return
                    elif i == 4: # Edge case
                        if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.vertices[i], ring.vertices[0], ring.vertices[1]}:
                            return
                    else:
                        if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.vertices[i], ring.vertices[i+1], ring.vertices[i+2]}:
                            return
            
            # Check the angle between the vectors is close to 108 degrees
            vec_theta = self.calc_vec_angle(vertex)
            if abs(vec_theta - 108) > FIVE_ANGLE_TOL:
                return
       
        # Check the angle between the planes formed by sets of 3 points, is less than 45 degrees, else don't continue
        if itter >= 3:
            plane_theta = self.calc_plane_angle(vertex)
            if plane_theta > PLANE_TILT_CUTOFF:
                return

        # Check if the sequence of 4 points (formed on the 3rd itteration) is closeable via a 5th point
        if itter == 3:
            # If any of the starting point's neighbours are neighbours of the current vertex's 3rd predecessor, the ring is closed
            for starting_point_neighbour in vertex.predecessor.predecessor.predecessor.neighbours:
                if vertex.is_neighbour(starting_point_neighbour):
                    # print("Close ring: " + str(starting_point_neighbour.index) + ": (" + str(starting_point_neighbour.x) + ", " + str(starting_point_neighbour.y) + ", " + str(starting_point_neighbour.z) + ")")
                    self.closeable = True
                    self.five_ring.append(starting_point_neighbour)
                    return
                
        if not self.closeable: # Only run if the ring has not been closed (to avoid uneccesary computation)
            # Loop through each neighbour of the current vertex 
            for neighbour in vertex.neighbours:
                # DFS
                if not neighbour.visited and itter < 4 and not self.closeable:
                    # The neighbour's predecessor is now the current vertex (used for backtracing)
                    neighbour.predecessor = vertex
                    # Check the second vertex in the sequence is 1.6 times the HBond distance from the starting point, R(i, i+2) = 1.6*a
                    if itter >= 2:
                        if abs(1.6*vertex.dist(neighbour) - neighbour.dist(neighbour.predecessor.predecessor)) < FIVE_RING_TOL:
                            self.fiveDFSRingFinder(neighbour.index, itter + 1) # Recursion
                            if self.closeable:
                                self.five_ring.append(neighbour) # Only add to the ring variable if it is closable            
                    else:
                        if neighbour.neighbours is not None: 
                            self.fiveDFSRingFinder(neighbour.index, itter + 1) # Recursion
                        if self.closeable:
                            self.five_ring.append(neighbour)

    ## Recurvise depth-first search for HEXAGONAL rings
    def sixDFSRingFinder(self, start_index, itter):
        # Get the vertex from the list of all verticies in the graph
        vertex = self.vertices[start_index]
        vertex.visited = True

        # Check if the current sequence of 3 points in already in a ring, if so, don't continue
        if itter >= 2:
            for ring in self.six_ring_list:
                for i in range(6):
                    if i == 4: # Edge case
                        if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.vertices[i], ring.vertices[i+1], ring.vertices[0]}:
                            return
                    elif i == 5: # Edge case
                        if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.vertices[i], ring.vertices[0], ring.vertices[1]}:
                            return
                    else:
                        if {vertex, vertex.predecessor, vertex.predecessor.predecessor} == {ring.vertices[i], ring.vertices[i+1], ring.vertices[i+2]}:
                            return
            
            # Check the angle between the vectors is close to 120 degrees
            vec_theta = self.calc_vec_angle(vertex)
            if abs(vec_theta - 120) > SIX_ANGLE_TOL:
                return
        
        # Check the angle between the planes formed by sets of 3 points, is less than 45 degrees, else don't continue
        if itter >= 3:
            plane_theta = self.calc_plane_angle(vertex)
            if plane_theta > PLANE_TILT_CUTOFF:
                return

        # Check if the sequence of 4 points (formed on the 3rd itteration) is closeable via a 5th point
        if itter == 4:
            # If any of the starting point's neighbours are neighbours of the current vertex's 3rd predecessor, the ring is closed
            for starting_point_neighbour in vertex.predecessor.predecessor.predecessor.predecessor.neighbours:
                if vertex.is_neighbour(starting_point_neighbour):
                    self.closeable = True
                    self.six_ring.append(starting_point_neighbour)
                    return
                
        if not self.closeable: # Only run if the ring has not been closed (to avoid uneccesary computation)
            # Loop through each neighbour of the current vertex 
            for neighbour in vertex.neighbours: 
                # print(itter)
                if not neighbour.visited and itter < 5 and not self.closeable:
                    # The neighbour's predecessor is now the current vertex (used for backtracing)
                    neighbour.predecessor = vertex
                    # Check the third vertex in the sequence is 2 times the HBond distance from the starting point R(i, i+2) = 1.6*a
                    if itter >= 3:
                        if abs(2*vertex.dist(neighbour) - neighbour.dist(neighbour.predecessor.predecessor.predecessor)) < SIX_RING_TOL:
                            self.sixDFSRingFinder(neighbour.index, itter + 1) # Recursion
                            if self.closeable:
                                self.six_ring.append(neighbour) # Only add to the ring variable if it is closable
                    else:
                        if neighbour.neighbours is not None:
                            self.sixDFSRingFinder(neighbour.index, itter + 1) # Recursion
                        if self.closeable:
                            self.six_ring.append(neighbour) # Only add to the ring if it is closeable

    # Function to calculate the angle between two vectors based on 3 consecutive points
    def calc_vec_angle(self, vertex):
        P = np.array([vertex.predecessor.predecessor.x, vertex.predecessor.predecessor.y, vertex.predecessor.predecessor.z])
        Q = np.array([vertex.predecessor.x, vertex.predecessor.y, vertex.predecessor.z])    
        R = np.array([vertex.x, vertex.y, vertex.z])

        QP = P - Q
        norm_QP = np.linalg.norm(QP)
        QR = R - Q
        norm_QR = np.linalg.norm(QR)

        theta = np.degrees(np.arccos(np.dot(QP, QR) / (norm_QP * norm_QR)))
        return theta

    # Function to calculate the angle between two planes based on 3 consecutive points
    def calc_plane_angle(self, vertex):
        P = np.array([vertex.predecessor.predecessor.predecessor.x, vertex.predecessor.predecessor.predecessor.y, vertex.predecessor.predecessor.predecessor.z])
        Q = np.array([vertex.predecessor.predecessor.x, vertex.predecessor.predecessor.y, vertex.predecessor.predecessor.z])    
        R = np.array([vertex.predecessor.x, vertex.predecessor.y, vertex.predecessor.z])
        S = np.array([vertex.x, vertex.y, vertex.z])

        PQ = Q - P
        PR = R - P
        normal_1 = np.cross(PQ, PR)
        norm_1 = np.linalg.norm(normal_1)

        PR = R - P
        PS = S - P
        normal_2 = np.cross(PR, PS)
        norm_2 = np.linalg.norm(normal_2)

        theta = np.degrees(np.arccos(np.dot(normal_1, normal_2) / (norm_1 * norm_2)))
        return theta

    # Reset all the verticies of the graph to default
    def clear(self):
        self.closeable = False # No longer closeable
        for vertex in self.vertices: 
            vertex.predeccesor = None # Remove vertext predecessor 'memory'
            vertex.visited = False # Reset visited flag

    def add_ring_to_matrix(self, start_index):
        # If we have found a ring (len = 4 since we don't add the start index to the list in DFS)
        if len(self.five_ring) == 4:
            self.five_ring.append(self.vertices[start_index]) # Add the starting O2 to the list
            self.five_ring_list.append(self.Ring(self.five_ring))
            self.five_ring = [] # Reset the ring list
            
        # If we have found a ring (len = 5 since we don't add the start index to the list in DFS)
        elif len(self.six_ring) == 5:
            self.six_ring.append(self.vertices[start_index])
            self.six_ring_list.append(self.Ring(self.six_ring))
            self.six_ring = [] # Reset the ring list
            
        # Reset the ring lists if no ring is found
        else:
            self.five_ring = []
            self.six_ring = []

    # NOTE: Could make this method and the next into a single method with more inputs
    def six_find_neighbouring_five_rings(self):
        # For each hexagonal ring, find neighbouring pentagonal rings
        for hex_ring in self.six_ring_list:
            for pent_ring in self.five_ring_list:  
                for i in range(6): 
                    for j in range(5):
                        if i == 5 and j == 4: # Edge case (loop around from the last vertex to the first vertex, both rings)
                            if {hex_ring.vertices[i], hex_ring.vertices[0]} == {pent_ring.vertices[j], pent_ring.vertices[0]}: 
                                hex_ring.neighbouring_rings.add(pent_ring)
                                pent_ring.neighbouring_rings.add(hex_ring)
                                hex_ring.coordination += 1
                                pent_ring.coordination += 1
                        elif i == 5: # Edge case (loop around from the last vertex to the first vertex, hexagon)
                            if {hex_ring.vertices[i], hex_ring.vertices[0]} == {pent_ring.vertices[j], pent_ring.vertices[j+1]}: 
                                hex_ring.neighbouring_rings.add(pent_ring)
                                pent_ring.neighbouring_rings.add(hex_ring)
                                hex_ring.coordination += 1
                                pent_ring.coordination += 1
                        elif j == 4: # Edge case (loop around from the last vertex to the first vertex, pentagon)
                            if {hex_ring.vertices[i], hex_ring.vertices[i+1]} == {pent_ring.vertices[j], pent_ring.vertices[0]}: 
                                hex_ring.neighbouring_rings.add(pent_ring)
                                pent_ring.neighbouring_rings.add(hex_ring)
                                hex_ring.coordination += 1
                                pent_ring.coordination += 1
                        else:
                            if {hex_ring.vertices[i], hex_ring.vertices[i+1]} == {pent_ring.vertices[j], pent_ring.vertices[j+1]}: 
                                hex_ring.neighbouring_rings.add(pent_ring)
                                pent_ring.neighbouring_rings.add(hex_ring)
                                hex_ring.coordination += 1
                                pent_ring.coordination += 1

    # Function to find all of the neighbours of each pentagonal ring
    def five_find_neighbouring_five_rings(self):
        # For each pentagonal ring, find neighbouring pentagonal rings
        for i in range(len(self.five_ring_list) - 1):
            for j in range(i + 1, len(self.five_ring_list)):
                for k in range(5):
                    for l in range(5):
                        if k == 4 and l == 4: # Edge case (loop around from the last vertex to the first vertex, both rings)
                            if {self.five_ring_list[i].vertices[k], self.five_ring_list[i].vertices[0]} == {self.five_ring_list[j].vertices[l], self.five_ring_list[j].vertices[0]}: 
                                self.five_ring_list[i].neighbouring_rings.add(self.five_ring_list[j])
                                self.five_ring_list[j].neighbouring_rings.add(self.five_ring_list[i])
                                self.five_ring_list[i].coordination += 1
                                self.five_ring_list[j].coordination += 1
                        elif k == 4: # Edge case (loop around from the last vertex to the first vertex, pentagon)
                            if {self.five_ring_list[i].vertices[k], self.five_ring_list[i].vertices[0]} == {self.five_ring_list[j].vertices[l], self.five_ring_list[j].vertices[l+1]}: 
                                self.five_ring_list[i].neighbouring_rings.add(self.five_ring_list[j])
                                self.five_ring_list[j].neighbouring_rings.add(self.five_ring_list[i])
                                self.five_ring_list[i].coordination += 1
                                self.five_ring_list[j].coordination += 1
                        elif l == 4: # Edge case (loop around from the last vertex to the first vertex, pentagon)
                            if {self.five_ring_list[i].vertices[k], self.five_ring_list[i].vertices[k+1]} == {self.five_ring_list[j].vertices[l], self.five_ring_list[j].vertices[0]}: 
                                self.five_ring_list[i].neighbouring_rings.add(self.five_ring_list[j])
                                self.five_ring_list[j].neighbouring_rings.add(self.five_ring_list[i])
                                self.five_ring_list[i].coordination += 1
                                self.five_ring_list[j].coordination += 1
                        else:
                            if {self.five_ring_list[i].vertices[k], self.five_ring_list[i].vertices[k+1]} == {self.five_ring_list[j].vertices[l], self.five_ring_list[j].vertices[l+1]}: 
                                self.five_ring_list[i].neighbouring_rings.add(self.five_ring_list[j])
                                self.five_ring_list[j].neighbouring_rings.add(self.five_ring_list[i])
                                self.five_ring_list[i].coordination += 1
                                self.five_ring_list[j].coordination += 1


    # Find 6_1_5_6 rings. 1 fully coordinated hexagon with 6 pentagons, each of the pentagons share an edge with the hexagon and 2 other pentagons
    def find_6_1_5_6_cups(self):
        for hex_ring in self.six_ring_list:
            if hex_ring.coordination >= 6: # If the hexagon is fully coordinated
                hex_ring.visited = True
                for neighbouring_ring in hex_ring.neighbouring_rings: # For each neighbouring pentagon
                    stack = [(neighbouring_ring, 0)] # Add the pentagon to the stack with a depth of 0
                    skip = False
                    # Itterative depth first search
                    while stack: 
                        current_ring, depth = stack.pop()
                        current_ring.visited = True
                        if not skip:
                            # Check if any sequence of 3 rings (depth 2) is common to an already found cup
                            if depth >= 2 and self.cup_6_1_5_6_list:
                                for cup_6_1_5_6 in self.cup_6_1_5_6_list:
                                    for i in range(1, len(cup_6_1_5_6.rings)):
                                        if i == 5: # Edge case
                                            if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_6_1_5_6.rings[i], cup_6_1_5_6.rings[i+1], cup_6_1_5_6.rings[1]}:
                                                skip = True
                                                break 
                                        elif i == 6: # Edge case
                                            if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_6_1_5_6.rings[i], cup_6_1_5_6.rings[1], cup_6_1_5_6.rings[2]}:
                                                skip = True
                                                break
                                        else:
                                            if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_6_1_5_6.rings[i], cup_6_1_5_6.rings[i+1], cup_6_1_5_6.rings[i+2]}:
                                                skip = True # Don't continue, we have already found this cup
                                                break
                                    if skip: # Break out of the exterior loop
                                        break
                            
                            if depth > 4: # At a depth of 5, we are on the 6th pentagon
                                if current_ring.predecessor.predecessor.predecessor.predecessor.predecessor in current_ring.neighbouring_rings:# Check if the 5th ring is the root, then we have a 6_1_5_12 cup
                                    self.cup_6_1_5_6_list.append(self.Cup([hex_ring,
                                                                            current_ring.predecessor.predecessor.predecessor.predecessor.predecessor, 
                                                                            current_ring.predecessor.predecessor.predecessor.predecessor, 
                                                                            current_ring.predecessor.predecessor.predecessor, 
                                                                            current_ring.predecessor.predecessor, 
                                                                            current_ring.predecessor,
                                                                            current_ring])) # Add the cup to the list
                                    break
                                    
                            for ring in current_ring.neighbouring_rings: # DFS
                                if not ring.visited and ring.is_neighbour_of(hex_ring): # Ring needs to be a neighbour of the hexagon
                                    if current_ring.predecessor is not None:
                                        if ring.neighbour_orientation(hex_ring) == current_ring.predecessor.neighbour_orientation(hex_ring): # Ring needs to be on the same side of the hexagon as the previous ring
                                            ring.predecessor = current_ring
                                            stack.append((ring, depth + 1)) # Add the ring to the stack with a depth of 1 greater than the current ring
                                    else:
                                        ring.predecessor = current_ring
                                        stack.append((ring, depth + 1))
                    # Clear all visited flags
                    for ring in self.five_ring_list:    
                        ring.clear()

    def find_5_6_cups(self):
        # Ignore all hexagons since they are not part of the 5_6 cups
        for hex_ring in self.six_ring_list:
            hex_ring.visited = True
        # For each pentagon
        for pent_ring in self.five_ring_list:
            if pent_ring.coordination >= 5: # If the pentagon is fully coordinated
                pent_ring.visited = True
                for neighbouring_ring in pent_ring.neighbouring_rings: # For each neighbouring pentagon
                    stack = [(neighbouring_ring, 0)] # Add the pentagon to the stack with a depth of 0
                    skip = False
                    # Itterative depth first search
                    while stack:
                        current_ring, depth = stack.pop()
                        current_ring.visited = True

                        if not skip:
                            # Check if any sequence of 3 rings (depth 2) is common to an already found cup
                            if depth >= 2 and self.cup_5_6_list:
                                for cup_5_6 in self.cup_5_6_list:
                                    for i in range(1, len(cup_5_6.rings)):
                                        if i == 4:
                                            if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_5_6.rings[i], cup_5_6.rings[i+1], cup_5_6.rings[1]}:
                                                skip = True
                                                break
                                        elif i == 5:
                                            if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_5_6.rings[i], cup_5_6.rings[1], cup_5_6.rings[2]}:
                                                skip = True
                                                break
                                        else:   
                                            if {current_ring, current_ring.predecessor, current_ring.predecessor.predecessor} == {cup_5_6.rings[i], cup_5_6.rings[i+1], cup_5_6.rings[i+2]}:
                                                skip = True
                                                break
                                    if skip: # Break out of the exterior loop
                                        break

                            if depth > 3:
                                if current_ring.predecessor.predecessor.predecessor.predecessor in current_ring.neighbouring_rings:
                                    self.cup_5_6_list.append(self.Cup([ pent_ring,
                                                                        current_ring.predecessor.predecessor.predecessor.predecessor,
                                                                        current_ring.predecessor.predecessor.predecessor,
                                                                        current_ring.predecessor.predecessor,
                                                                        current_ring.predecessor,
                                                                        current_ring]))
                                    break
                        
                            for ring in current_ring.neighbouring_rings:
                                if not ring.visited and ring.is_neighbour_of(pent_ring):
                                    if current_ring.predecessor is not None:
                                        if ring.neighbour_orientation(pent_ring) == current_ring.predecessor.neighbour_orientation(pent_ring):
                                            ring.predecessor = current_ring
                                            stack.append((ring, depth + 1))
                                    else:
                                        ring.predecessor = current_ring
                                        stack.append((ring, depth + 1))

                    for ring in self.five_ring_list:
                        ring.clear()
        # Reset all hexagons
        for hex_ring in self.six_ring_list:
            hex_ring.visited = False

    # TODO: Make function to find 6_4_5_12 cages
    def find_6_4_5_12_cages(self):

        ### WAY TOO SLOW ###
        # For a cage to be formed, all combination of 2 cups should share 2 pentagons 
        length = sum(1 for ignore in combinations(self.cup_6_1_5_6_list, 4))
        # print(length)
        for combo in combinations(self.cup_6_1_5_6_list, 4):
            shared_in_combo = 0
            skip = False
            for cup in combo:
                common_rings = 0
                if not skip:
                    for other_cup in combo:
                        if cup != other_cup: 
                            if abs(cup.base_centroid().dist(other_cup.base_centroid()) - CAGE_6_4_5_12_ADJ_HEX_DIST) < CAGE_6_4_5_12_ADJ_HEX_DIST_TOL:
                                for ring in cup.rings:
                                    if ring in other_cup.rings:
                                        common_rings += 1
                            else:
                                skip = True
                                break
                        if common_rings == 2:
                            shared_in_combo += 1
                if skip:
                    break
                print(shared_in_combo)
                if shared_in_combo == 7:
                    self.cage_5_12_list.append(self.Cage(combo))

        # for i in range(len(self.cup_6_1_5_6_list) - 3): # Cup 1
        #     common_rings = 0
        #     for j in range(i, len(self.cup_6_1_5_6_list) - 2): # Cup 2
        #         # print(self.cup_6_1_5_6_list[i].base_centroid().dist(self.cup_6_1_5_6_list[j].base_centroid()))
        #         if abs(self.cup_6_1_5_6_list[i].base_centroid().dist(self.cup_6_1_5_6_list[j].base_centroid()) - CAGE_6_4_5_12_ADJ_HEX_DIST) < CAGE_6_4_5_12_ADJ_HEX_DIST_TOL:
        #             for ring in self.cup_6_1_5_6_list[i].rings:
        #                 if ring in self.cup_6_1_5_6_list[j].rings:
        #                     common_rings += 1
                    
                                
        #             for k in range(j, len(self.cup_6_1_5_6_list) - 1): # Cup 3
        #                 if abs(self.cup_6_1_5_6_list[i].base_centroid().dist(self.cup_6_1_5_6_list[j].base_centroid()) - CAGE_6_4_5_12_ADJ_HEX_DIST) < CAGE_6_4_5_12_ADJ_HEX_DIST_TOL:
        #                     for ring_1 in self.cup_6_1_5_6_list[j].rings:
        #                         if ring in self.cup_6_1_5_6_list[k].rings:
        #                             common_rings += 1
                                                
        #                 for l in range(k, len(self.cup_6_1_5_6_list)): # Cup 4
        #                     if abs(self.cup_6_1_5_6_list[i].base_centroid().dist(self.cup_6_1_5_6_list[j].base_centroid()) - CAGE_6_4_5_12_ADJ_HEX_DIST) < CAGE_6_4_5_12_ADJ_HEX_DIST_TOL:
        #                         for ring_1 in self.cup_6_1_5_6_list[k].rings:
        #                             if ring in self.cup_6_1_5_6_list[l].rings:
        #                                 common_rings += 1

        #                     if common_rings == 8:
        #                         self.cage_6_4_5_12_list.append(self.Cage([self.cup_6_1_5_6_list[i], self.cup_6_1_5_6_list[j], self.cup_6_1_5_6_list[k], self.cup_6_1_5_6_list[l]]))
        #                         break
        #     print(common_rings)
                                    
         
        ### DOESN'T WORK ###
        # for combo in combinations(self.cup_6_1_5_6_list, 4):
        #     flag = False

        #     vertex_list = []
        #     for cup in combo:
        #         for ring in cup.rings:
        #             for vertex in ring.verticies:
        #                 vertex_list.append(vertex)
        #     print(vertex_list)
        #     unique_verticies = set(vertex_list)
        #     num_verticies = len(unique_verticies)
        #     print(num_verticies)

        #     if num_verticies != 72:
        #         flag = True
            
        #     if not flag:
        #         self.cage_6_4_5_12_list.append(self.Cage(combo))

            # sum_x = 0
            # sum_y = 0
            # sum_z = 0
            # for verticies in unique_verticies:
            #     sum_x += verticies.x
            #     sum_y += verticies.y
            #     sum_z += verticies.z
            
            # average_x = sum_x / num_verticies
            # average_y = sum_y / num_verticies
            # average_z = sum_z / num_verticies

            # O = np.array([average_x, average_y, average_z])
            # print(O)

            # for cup in combo:
            #     normal = np.array(cup.oriented_normal)
            #     centroid = cup.base_centroid()
            #     centroid = np.array([centroid.x, centroid.y, centroid.z])
            #     centroid = centroid / np.linalg.norm(centroid) # Normalize centroid

            #     OG = O - centroid
            #     OG = OG / np.linalg.norm(OG) # Normalize OG

            #     theta_to_center = np.degrees(np.arccos(np.dot(normal, (-1)*OG) / (np.linalg.norm(normal) * np.linalg.norm((-1)*OG))))
            #     print(theta_to_center)

            #     if theta_to_center > CAGE_6_4_5_12_ANGLE_TOL:
            #         flag = True
            #         break

            # if not flag:
            #     for cup in combo:
            #         common_rings = 0
            #         for other_cup in combo:
            #             if cup != other_cup:
            #                 for ring in cup.rings:
            #                     if ring in other_cup.rings:
            #                         common_rings += 1
            #         print(common_rings)
                

    # Finds 6_2_5_12 cages. Cages are two cups, where each of the pentagonal rings on one cup have 2 neighbouring pentagonal rings on another cup
    def find_6_2_5_12_cages(self):
        for i in range(len(self.cup_6_1_5_6_list) - 1):
            cup_1 = self.cup_6_1_5_6_list[i]
            for j in range(i, len(self.cup_6_1_5_6_list)):
                common_edge_count = 0
                cup_2 = self.cup_6_1_5_6_list[j]
                # print(np.dot(cup_1.oriented_normal, (-1)*cup_2.oriented_normal))
                if np.isclose(np.dot(cup_1.oriented_normal, (-1)*cup_2.oriented_normal), 1, rtol=CAGE_6_2_5_12_BASE_TILT_TOL) and cup_1.base != cup_2.base:
                    # print(cup_1.base_centroid().dist(cup_2.base_centroid()))
                    if abs(cup_1.base_centroid().dist(cup_2.base_centroid()) - CAGE_6_2_5_12_HEIGHT) < CAGE_6_2_5_12_HEIGHT_TOL:
                        for k in range(1, len(cup_1.rings)):
                            for j in range(1, len(cup_2.rings)):
                                if cup_1.rings[k].is_neighbour_of(cup_2.rings[j]):
                                    common_edge_count += 1
                        # print(common_edge_count)
                        if common_edge_count == 12:
                            self.cage_6_2_5_12_list.append(self.Cage([cup_1, cup_2]))     


    def find_5_12_cages(self):
        for i in range(len(self.cup_5_6_list) - 1):
            cup_1 = self.cup_5_6_list[i]
            for j in range(i, len(self.cup_5_6_list)):
                common_edge_count = 0
                cup_2 = self.cup_5_6_list[j]
                if np.isclose(np.dot(cup_1.oriented_normal, (-1)*cup_2.oriented_normal), 1, rtol=CAGE_5_12_BASE_TILT_TOL) and set(cup_1.rings) != set(cup_2.rings):
                    if abs(cup_1.base_centroid().dist(cup_2.base_centroid()) - CAGE_5_12_HEIGHT) < CAGE_5_12_HEIGHT_TOL:
                        for k in range(1, len(cup_1.rings)):
                            for j in range(1, len(cup_2.rings)):
                                if cup_1.rings[k].is_neighbour_of(cup_2.rings[j]):
                                    common_edge_count += 1
                        # print(common_edge_count)
                        if common_edge_count == 10:
                            self.cage_5_12_list.append(self.Cage([cup_1, cup_2]))                                 

    def print_rings(self):
        # Print all rings
        count = 0
        for ring in self.five_ring_list:
            count += 1
            print("PENT RING " + str(count))
            for vertex in ring.vertices:
                print(str(vertex.index) + ": (" + str(vertex.x) + ", " + str(vertex.y) + ", " + str(vertex.z) + ")")
            print("\n")

        count = 0
        for ring in self.six_ring_list:
            count += 1
            print("HEX RING " + str(count))
            for vertex in ring.vertices:
                print(str(vertex.index) + ": (" + str(vertex.x) + ", " + str(vertex.y) + ", " + str(vertex.z) + ")")
            print("\n")

    def plot_rings(self):
        fig = plt.figure(figsize=plt.figaspect(0.5))
        ax = fig.add_subplot(1, 1, 1, projection='3d')
        ax.set_box_aspect(aspect_ratio) 
        # ax.set_proj_type('ortho')  # Use orthographic projection for faster rendering

        for ring in self.five_ring_list:
            vertices = np.array([[vertex.x, vertex.y, vertex.z] for vertex in ring.vertices])
            ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2],
                        marker='o',
                        s=100,
                        c='r',
                        alpha=1.0)  # Set alpha to 1.0 for solid colors
        
        for ring in self.six_ring_list:
            vertices = np.array([[vertex.x, vertex.y, vertex.z] for vertex in ring.vertices])
            ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2],
                        marker='o',
                        s=100,
                        c='b',
                        alpha=1.0)  # Set alpha to 1.0 for solid colors
        plt.show()

    def plot_these_rings(self, rings):
        fig = plt.figure(figsize=plt.figaspect(0.5))
        ax = fig.add_subplot(1, 1, 1, projection='3d')
        ax.set_box_aspect(aspect_ratio) 
        # ax.set_proj_type('ortho')  # Use orthographic projection for faster rendering

        for ring in rings:
            vertices = np.array([[vertex.x, vertex.y, vertex.z] for vertex in ring.vertices])
            ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2],
                        marker='o',
                        s=100,
                        c='r')
        plt.show()
     
    def print_cups(self):
        # Print all cups
        count = 0
        for cup in self.cup_6_1_5_6_list:
            count += 1
            print("CUP " + str(count))
            for ring in cup.rings:
                ring.print_ring()
            print("\n")

    def plot_cups(self):
        fig = plt.figure(figsize=plt.figaspect(0.5))
        ax = fig.add_subplot(1, 1, 1, projection='3d')
        ax.set_box_aspect(aspect_ratio) 
        ax.set_proj_type('ortho')  # Use orthographic projection for faster rendering
        for cup in self.cup_6_1_5_6_list:
            for ring in cup.rings:
                vertices = np.array([[vertex.x, vertex.y, vertex.z] for vertex in ring.vertices])
                ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2],
                            marker='o',
                            s=100,
                            c='r')
            base_vertices = np.array([[vertex.x, vertex.y, vertex.z] for vertex in cup.base.vertices])
            ax.scatter(base_vertices[:, 0], base_vertices[:, 1], base_vertices[:, 2],
                        marker='o',
                        s=100,
                        c='b')
        plt.show()

    def plot_these_cups(self, cups):
            fig = plt.figure(figsize=plt.figaspect(0.5))
            ax = fig.add_subplot(1, 1, 1, projection='3d')

            # Plot all atoms
            for cup in cups:
                for ring in cup.rings:
                    for vertex in ring.vertices:
                        ax.scatter(vertex.x, vertex.y, vertex.z,
                                marker='o',
                                s=100,
                                c='r')
                for vertex in cup.base.vertices:
                    ax.scatter(vertex.x, vertex.y, vertex.z,
                                marker='o',
                                s=100,
                                c='b')

    def print_6_2_5_12_cages(self):
        # Print all cages
        count = 0
        for cage in self.cage_6_2_5_12_list:
            count += 1
            print("CAGE " + str(count))
            cage.print_cage()
            cage.plot_cage()
    
    def plot_6_2_5_12_cages(self):
        fig = plt.figure(figsize=plt.figaspect(0.5))
        ax = fig.add_subplot(1, 1, 1, projection='3d')
        ax.set_box_aspect(aspect_ratio) 
        ax.set_proj_type('ortho')  # Use orthographic projection for faster rendering

        for cage in self.cage_6_2_5_12_list:
            for cup in cage.cups:
                for ring in cup.rings:
                    vertices = np.array([[vertex.x, vertex.y, vertex.z] for vertex in ring.vertices])
                    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2],
                               marker='o',
                               s=100,
                               c='r')
                base_vertices = np.array([[vertex.x, vertex.y, vertex.z] for vertex in cup.base.vertices])
                ax.scatter(base_vertices[:, 0], base_vertices[:, 1], base_vertices[:, 2],
                           marker='o',
                           s=100,
                           c='b')
        plt.show()  

    def plot_6_4_5_12_cages(self):
        fig = plt.figure(figsize=plt.figaspect(0.5))
        ax = fig.add_subplot(1, 1, 1, projection='3d')
        ax.set_box_aspect(aspect_ratio) 
        ax.set_proj_type('ortho')  # Use orthographic projection for faster rendering

        for cage in self.cage_6_4_5_12_list:
            for cup in cage.cups:
                for ring in cup.rings:
                    vertices = np.array([[vertex.x, vertex.y, vertex.z] for vertex in ring.vertices])
                    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2],
                               marker='o',
                               s=100,
                               c='r')
                base_vertices = np.array([[vertex.x, vertex.y, vertex.z] for vertex in cup.base.vertices])
                ax.scatter(base_vertices[:, 0], base_vertices[:, 1], base_vertices[:, 2],
                           marker='o',
                           s=100,
                           c='b')
        plt.show()  

    class Cage():
        def __init__(self, cups):
            self.cups = cups

        def print_cage(self):
            print("CAGE")
            for cup in self.cups:
                cup.print_cup()
            print("\n")

        def plot_cage(self):
            fig = plt.figure(figsize=plt.figaspect(0.5))
            ax = fig.add_subplot(1, 1, 1, projection='3d')
            ax.set_box_aspect(aspect_ratio)  
            ax.set_proj_type('ortho')  # Use orthographic projection for faster rendering

            for cup in self.cups:
                for ring in cup.rings:
                    vertices = np.array([[vertex.x, vertex.y, vertex.z] for vertex in ring.vertices])
                    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2],
                               marker='o',
                               s=100,
                               c='r')
                base_vertices = np.array([[vertex.x, vertex.y, vertex.z] for vertex in cup.base.vertices])
                ax.scatter(base_vertices[:, 0], base_vertices[:, 1], base_vertices[:, 2],
                           marker='o',
                           s=100,
                           c='b')
            plt.show()

    class Cup():
        def __init__(self, rings):
            # List of 1 hexagon (frist) and 6 pentagons for 6_1_5_6 cup
            # List of 1 pentagon (frist) and 5 pentagons for 5_6 cup
            self.rings = rings 
            self.base = rings[0]
            
            # Find the normal of the cup oriented with respect to its neighbours
            self.oriented_normal = self.rings[1].neighbour_orientation(self.base, return_oriented_normal=True)

        def base_centroid(self):
            x = 0
            y = 0
            z = 0
            count = 0
            for vertex in self.base.vertices:
                count += 1
                x += vertex.x
                y += vertex.y
                z += vertex.z
            return Graph.Vertex(0, x/count, y/count, z/count)
         
        def print_cup(self):
            print("CUP")
            for ring in self.rings:
                ring.print_ring()
            print("\n")

        # Makes a new plot of just the atoms in the cup
        def plot_cup(self):
            fig = plt.figure(figsize=plt.figaspect(0.5))
            ax = fig.add_subplot(1, 1, 1, projection='3d')

            # Plot all atoms
            for ring in self.rings:
                for vertex in ring.vertices:
                    ax.scatter(vertex.x, vertex.y, vertex.z,
                            marker='o',
                            s=100,
                            c='r') # Red for the pentagons
            for vertex in self.base.vertices:
                ax.scatter(vertex.x, vertex.y, vertex.z,
                            marker='o',
                            s=100,
                            c='b') # blue for the hexagon base
            plt.show()

    class Ring():
        def __init__(self, vertices):
            self.vertices = vertices
            self.num_vertices = len(vertices)
            self.neighbouring_rings = set()
            self.coordination = 0
            self.visited = False
            self.predecessor = None

        # Finds and returns which side of the plane of the hexagon this ring is on.
        def neighbour_orientation(self, base_ring, return_oriented_normal=False):
            P = np.array([base_ring.vertices[1].x, base_ring.vertices[1].y, base_ring.vertices[1].z])
            Q = np.array([base_ring.vertices[2].x, base_ring.vertices[2].y, base_ring.vertices[2].z])
            R = np.array([base_ring.vertices[0].x, base_ring.vertices[0].y, base_ring.vertices[0].z])
            hex_vec_1 = Q - P
            hex_vec_2 = R - P
            hex_normal = np.cross(hex_vec_1, hex_vec_2)

            for vertex in self.vertices:
                if vertex not in base_ring.vertices:
                    N = np.array([vertex.x, vertex.y, vertex.z])
                    if return_oriented_normal:
                        return (hex_normal * np.sign(np.dot(hex_normal, N - P)))/np.linalg.norm(hex_normal)
                    return np.sign(np.dot(hex_normal, N - P))
            
        def print_neighbouring_rings(self):
            count = 0
            for ring in self.neighbouring_rings:
                count += 1
                print("NEIGHBOURING RING: " + str(count))
                for vertex in ring.vertices:
                    print(str(vertex.index) + ": (" + str(vertex.x) + ", " + str(vertex.y) + ", " + str(vertex.z) + ")")
                print("\n")

        def is_neighbour_of(self, ring):
            if self in ring.neighbouring_rings:
                return True

        def clear(self):
            self.visited = False
            self.predecessor = None
        
        def equals(self, ring):
            if set(self.vertices) == set(ring.vertices):
                return True
            return False
        
        def print_ring(self):
            for vertex in self.vertices:
                print(str(vertex.index) + ": (" + str(vertex.x) + ", " + str(vertex.y) + ", " + str(vertex.z) + ")")
            print("\n") 

        def plot_ring(self):
            fig = plt.figure(figsize=plt.figaspect(0.5))
            ax = fig.add_subplot(1, 1, 1, projection='3d')
            
            ax.set_xlim([0, 10])
            ax.set_ylim([0, 10])
            ax.set_zlim([0, 10])

            for vertex in self.vertices:
                ax.scatter(vertex.x, vertex.y, vertex.z,
                            marker='o',
                            s=100,
                            c='r')
            plt.show()

    class Vertex:
        def __init__(self, i, x, y, z):
            self.index = i # Particle index in the xyz file 
            self.x = x 
            self.y = y
            self.z = z
            
            self.visited = False # Visited var
            
            self.neighbours = [] # List of first neighbours (array of Vertex objects)
            self.num_neighbours = 0
            
            self.predecessor = None # Predecessor for backtracking

        def populate_neighbours(self, vertices): # Takes in a list of verticies
            count = 0
            # For every vertex, find its neighbours
            for vertex in vertices:
                if self == vertex:
                    continue
                if self.is_neighbour(vertex):
                    # self.neighbours.add(vertex)
                    self.neighbours.append(vertex)
                    count += 1
            # random.shuffle(self.neighbours) # Avoids bias in DFS (trying to avoid the same path being taken from the same vertex ... I think)
            self.num_neighbours = count
        
        def is_neighbour(self, Q):
            # If the current vertex (self) and the vertex Q are the toleranced HBond distance apart, they are neighbours
            if (abs(self.dist(Q) - HBOND_DIST) < HBOND_DIST_TOL):
                return True 
            return False 

        def dist(self, Q):
            # Simple euclidean distance calculation
            return np.sqrt((self.x-Q.x)**2 + (self.y-Q.y)**2 + (self.z-Q.z)**2) 
        

## Graph Initialisation

To initialize the graph object, pass the dataframe of oxygen atoms define earlier. This may take several seconds as the neighbour lists of each vertex are populated.

In [ ]:
# Initialise the graph
g = Graph(oxygen_df)

/tmp/ipykernel_75390/2966412463.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vertices.iloc[i][1],
/tmp/ipykernel_75390/2966412463.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vertices.iloc[i][2],
/tmp/ipykernel_75390/2966412463.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vertices.iloc[i][3]


## Execution

The entire identification procedure is handled by calling the find_all() method on the initialized graph method. This identifies all of the 5-membered rings, 6-membered rings, 6<sup>1</sup>5<sup>6</sup> cups, 5<sup>6</sup> cups, 6<sup>2</sup>5<sup>12</sup> cages and 5<sup>12</sup> cages. To change the execution, adapt the find_all() method in the Graph class. 

The summary() Graph method outputs the reults of the find_all() procedure in text format.

<u> REINITIALIZE THE GRAPH BETWEEN SEARCHES</u>

In [ ]:
# Find all rings
g.find_all()
g.summary()


Summary:
Rings:
	 5 membered rings: 963
	 6 membered rings: 101
Cups:
	 6_1_5_12 cups: 190
	 5_6 cups: 1309
Cages:
	 6_2_5_12 cages: 76
	 5_12 cages: 168


## The Output File

Below is a function to create an .xyz file with the results of the search procedure. This not only outputs the oxygen atoms of the identified cages, but also their corresponding hydrogen atoms to form water molecules. Thus, a hydrogen dataframe must be passed to the function. An output filename must be specified aswell as what is to be output via the out_type parameter.  

In [ ]:
def make_output_file(filename, graph, h_df, out_type="rings"):
    DELTA = 1.0 # Distance tolerance between H and O atoms of a water molecule

    # Add the file extension if it is not already
    if not filename.endswith(".xyz"):
        filename += ".xyz"
    # Create a list to store the atoms
    match out_type:
        case "rings":
            atoms = []
            for ring in graph.five_ring_list:
                for vertex in ring.vertices:
                    atoms.append(["O", vertex.x, vertex.y, vertex.z])
                    for i, H in enumerate(h_df.values):
                        idx_to_remove = []
                        hydrogen = H[1:].tolist()
                        if dist([vertex.x, vertex.y, vertex.z], hydrogen) < DELTA:
                            atoms.append(["H", hydrogen[0], hydrogen[1], hydrogen[2]])
                            idx_to_remove.append(i)
            for ring in graph.six_ring_list:
                for vertex in ring.vertices:
                    atoms.append(["O", vertex.x, vertex.y, vertex.z])
                    for i, H in enumerate(h_df.values):
                        idx_to_remove = []
                        hydrogen = H[1:].tolist()
                        if dist([vertex.x, vertex.y, vertex.z], hydrogen) < DELTA:
                            atoms.append(["H", hydrogen[0], hydrogen[1], hydrogen[2]])
                            idx_to_remove.append(i)
        case "6_2_5_6 cups":
            atoms = []
            for cup in graph.cup_6_1_5_6_list:
                for ring in cup.rings:
                    for vertex in ring.vertices:
                        atoms.append(["O", vertex.x, vertex.y, vertex.z])
                        for i, H in enumerate(h_df.values):
                            idx_to_remove = []
                            hydrogen = H[1:].tolist()
                            if dist([vertex.x, vertex.y, vertex.z], hydrogen) < DELTA:
                                atoms.append(["H", hydrogen[0], hydrogen[1], hydrogen[2]])
                                idx_to_remove.append(i)
        case "5_6 cups":
            atoms = []
            for cup in graph.cup_5_6_list:
                for ring in cup.rings:
                    for vertex in ring.vertices:
                        atoms.append(["O", vertex.x, vertex.y, vertex.z])
                        for i, H in enumerate(h_df.values):
                            idx_to_remove = []
                            hydrogen = H[1:].tolist()
                            if dist([vertex.x, vertex.y, vertex.z], hydrogen) < DELTA:
                                atoms.append(["H", hydrogen[0], hydrogen[1], hydrogen[2]])
                                idx_to_remove.append(i)     
        case "6_2_5_12 cages":
            atoms = []
            for cage in graph.cage_6_2_5_12_list:
                for cup in cage.cups:
                    for ring in cup.rings:
                        for vertex in ring.vertices:
                            atoms.append(["O", vertex.x, vertex.y, vertex.z])
                            for i, H in enumerate(h_df.values):
                                idx_to_remove = []
                                hydrogen = H[1:].tolist()
                                if dist([vertex.x, vertex.y, vertex.z], hydrogen) < DELTA:
                                    atoms.append(["H", hydrogen[0], hydrogen[1], hydrogen[2]])
                                    idx_to_remove.append(i)
                        
                            h_df.drop(idx_to_remove, inplace=True, errors='ignore')    
        case "5_12 cages":
            atoms = []
            for cage in graph.cage_5_12_list:
                for cup in cage.cups:
                    for ring in cup.rings:
                        for vertex in ring.vertices:
                            atoms.append(["O", vertex.x, vertex.y, vertex.z])
                            for i, H in enumerate(h_df.values):
                                idx_to_remove = []
                                hydrogen = H[1:].tolist()
                                if dist([vertex.x, vertex.y, vertex.z], hydrogen) < DELTA:
                                    atoms.append(["H", hydrogen[0], hydrogen[1], hydrogen[2]])
                                    idx_to_remove.append(i)
                        
                            h_df.drop(idx_to_remove, inplace=True, errors='ignore')

    atoms = [list(atom) for atom in set(tuple(atom) for atom in atoms)]
    # H_atoms = h_df.values.tolist()
    # num_atoms = len(atoms) + len(h_df)
    num_atoms = len(atoms)

    # Write the output to a file
    with open(filename, 'w') as file:
        file.write(f"{num_atoms}\n") 
        # How do I find the lattic constant? Do I even need it?
        file.write(f"sI: {num_atoms} water molecules, lattice constant a = 12.03\n")
        
        # for vertex in H_atoms:
        #     file.write(f"{vertex[0]} {vertex[1]} {vertex[2]} {vertex[3]}\n")

        for vertex in atoms:
            file.write(f"{vertex[0]} {vertex[1]} {vertex[2]} {vertex[3]}\n")
        

In [ ]:
make_output_file("test.xyz", g, hydrogen_df, out_type="6_2_5_12 cages")

/tmp/ipykernel_75390/433719129.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_df.drop(idx_to_remove, inplace=True, errors='ignore')
/tmp/ipykernel_75390/433719129.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_df.drop(idx_to_remove, inplace=True, errors='ignore')
/tmp/ipykernel_75390/433719129.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_df.drop(idx_to_remove, inplace=True, errors='ignore')
/tmp/ipykernel_7539

In [ ]:
# # Find neighbouring pentagonal rings for each hexagonal ring
# g.six_find_neighbouring_five_rings()

# # Find neighbouring pentagonal rings for each pentagonal ring
# g.five_find_neighbouring_five_rings() 


In [ ]:
# ring_index = 1

# g.six_ring_list[ring_index].print_neighbouring_rings()

In [ ]:
# g.find_6_1_5_6_cups()

In [ ]:
# print("Num Cups:" + str(len(g.cup_6_1_5_6_list)))

In [ ]:
# list(g.six_ring_list[1].neighbouring_rings)[5].neighbour_orientation(g.six_ring_list[1])

In [ ]:
g.six_ring_list[0].coordination

12

In [ ]:
# g.cup_6_1_5_6_list[4].plot_cup()

In [ ]:
# g.find_6_2_5_12_cages()
# print("Num Cages: " + str(len(g.cage_6_2_5_12_list)))

In [ ]:
# g.cage_6_2_5_12_list[0].plot_cage()

In [ ]:
# g.cup_6_1_5_6_list[0].base_centroid().dist(g.cup_6_1_5_6_list[2].base_centroid())

In [ ]:
six_idx = 0
five_idx = 1

# g.six_ring_list[six_idx].print_ring()
# g.six_ring_list[six_idx].plot_ring()

# g.five_ring_list[five_idx].print_ring()
# g.five_ring_list[five_idx].plot_ring()

# g.plot_these_rings([g.six_ring_list[six_idx], g.five_ring_list[five_idx]])
g.plot_these_rings(g.six_ring_list[0].neighbouring_rings)

# angle_plane([g.five_ring_list[idx].verticies[0].x, g.five_ring_list[idx].verticies[0].y, g.five_ring_list[idx].verticies[0].z],
#             [g.five_ring_list[idx].verticies[1].x, g.five_ring_list[idx].verticies[1].y, g.five_ring_list[idx].verticies[1].z],
#             [g.five_ring_list[idx].verticies[2].x, g.five_ring_list[idx].verticies[2].y, g.five_ring_list[idx].verticies[2].z],
#             [g.five_ring_list[idx].verticies[3].x, g.five_ring_list[idx].verticies[3].y, g.five_ring_list[idx].verticies[3].z])


<IPython.core.display.Javascript object>

In [ ]:
g.plot_rings()

<IPython.core.display.Javascript object>

In [ ]:
g.plot_cups()

<IPython.core.display.Javascript object>

In [ ]:
idx_1 = 2
idx_2 = 3
idx_3 = 5
l1 = [g.cup_6_1_5_6_list[idx_1].rings[i].vertices[j] for i in range(len(g.cup_6_1_5_6_list[idx_1].rings)) for j in range(len(g.cup_6_1_5_6_list[idx_1].rings[i].vertices))]
l2 = [g.cup_6_1_5_6_list[idx_2].rings[i].vertices[j] for i in range(len(g.cup_6_1_5_6_list[idx_2].rings)) for j in range(len(g.cup_6_1_5_6_list[idx_2].rings[i].vertices))]
lf = set(l1 + l2)
print(len(lf))
g.plot_these_cups([g.cup_6_1_5_6_list[idx_1], g.cup_6_1_5_6_list[idx_2], g.cup_6_1_5_6_list[idx_3]])

36


<IPython.core.display.Javascript object>

In [ ]:
g.cage_6_4_5_12_list[0].plot_cage()

AttributeError: 'Graph' object has no attribute 'cage_6_4_5_12_list'

In [ ]:
g.plot_6_4_5_12_cages()

<IPython.core.display.Javascript object>

AttributeError: 'Graph' object has no attribute 'cage_6_4_5_12_list'